<a id='toc'></a>

# Medical Appointment No Shows - What is going on?

One of the limitations I've faced breaking into machine learning domain, which is actually not a limitation rather a blessing, is the overwhelming vast amount of knowledge and resources available leaving me puzzled where to start and what topics to cover.

So I've decided to create this notebook with that struggle in mind; displaying a structured work flow of an imbalanced binary classification problem but with a twist! There are a lot of notebooks that shows how to do things right, but few that discusses how things can actually go wrong, spotting errors and their impact on final results with examples.

We'll cover many topics that are important to be aware of going through a classification problem, I came across these topics during my efforts to improve on my first classification attempt going through different research papers, books, blogs and other notebooks; I wanted to make this notebook as a collective reference for such topics and at the same time provide links to external sources that I've found to be very informative and well presented.

I have a particular interest in this dataset, I've chosen it to be my first EDA project in python language then to be my first hands on applied machine learning project and now I'm providing updates to my first classification attempt using new knowledge acquired. Each section in this notebook was created at different time interval; I'm keeping them intact for the joy of reflecting back on my previous work and measuring progress.

The three main sections below are sorted chronologically:
- Exploratory Data Analysis (EDA)
- Predictions (1st attempt)
- Updates

## Table of Contents

- About the dataset and basic preprocessing
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
</ul>  

- Exploratory Data Analysis (EDA)
<ul>
<li><a href="#eda">Exploratory Data Analysis (EDA)</a></li>
<li><a href="#conclusions">EDA - Conclusion and limitations</a></li>
</ul>

- Predictions (1st attempt)
<ul>
<li><a href="#predictions">Appointment predictions</a></li>
<li><a href="#predconc">Predictions conclusion - Who's To Blame?</a></li>
</ul>

- Updates
<ul>
<li><a href="#analysis">In-depth analysis and feature engineering</a></li>
    <ul>
    <li><a href="#panalysis">Analysing patient behavior</a></li>
    <li><a href="#fengn">Numeric feature engineering</a></li>
    <li><a href="#bline">Establishing baseline and analysing results</a></li>
    <li><a href="#fengc">Categorical feature engineering</a></li>
    <li><a href="#pm">Panic Attack!</a></li>
    </ul>
<li><a href="#modeling">Modeling</a></li>
    <ul>
    <li><a href="#fs">Feature selection</a></li>
    <li><a href="#rts">Resampling training set</a></li>
    <li><a href="#ovrft">Overfitting</a></li>
    <li><a href="#thrm">Threshold moving</a></li>
    <li><a href="#kfcv">K-fold cross validation</a></li>
    <li><a href="#vdb">Visualizing decision boundary</a></li>
    <li><a href="#mfnc">More features and a new classifier</a></li>
    <li><a href="#upr">Updated prediction results: RandomForest and CatBoost</a></li>
    <li><a href="#pm2">Heart skips a beat!</a></li>
    <li><a href="#rvapp">Revised Approach</a></li>
    </ul>
</ul>

- Wrap-up
<ul>
<li><a href="#references">Summary of updates and additional references</a></li>
</ul>

In [6]:
# pip install pyod

In [7]:
# Load libraries

import pandas as pd
import numpy as np
import datetime as dt
from time import time
from collections import Counter
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly import subplots
import squarify
from sklearn.dummy import DummyClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2, f_classif, mutual_info_classif, SelectFromModel
from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import precision_recall_curve, roc_curve, auc, classification_report, confusion_matrix, f1_score
from sklearn.decomposition import PCA
from sklearn.manifold import MDS,TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, RobustScaler, MaxAbsScaler, LabelEncoder
import category_encoders as ce
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pyod.models.pca import PCA as PCA_pyod
import catboost as cb
import catboost.utils as utils

# # setting pandas display options
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', 2000)
# pd.set_option('display.float_format', '{:20,.2f}'.format)
# pd.set_option('display.max_colwidth', None)

# # reset display options
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.width')
# pd.reset_option('display.float_format')
# pd.reset_option('display.max_colwidth')

%matplotlib inline
# from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode(connected=True)

<a id='intro'></a>
## Introduction

The dataset includes medical appointment status as well as some patient details of Brazilian families; I'd be interested in investigating the reason why some patients do not show up to their scheduled appointments and whether there are opportunities for improvement in appointment administration that would result in a higher attendance frequency.

The dataset consists of 14 variables as follows:

1. PatientId : Unique identification of a patient
2. AppointmentID : Unique identification of each appointment
3. Gender: Male or Female.
4. ScheduledDay: The day of registering the appointment.
5. AppointmentDay: The day of actual appointment.
6. Age: How old is the patient.
7. Neighbourhood: Where the appointment takes place.
8. Scholarship: Whether the patient is enrolled in [Bolsa_Família](https://en.wikipedia.org/wiki/Bolsa_Fam%C3%ADlia), which is a social welfare program of the Government of Brazil
9. Hipertension: Hypertension, also known as high blood pressure. Part of patient's medical history.
10. Diabetes: Part of patient's medical history.
11. Alcoholism: drinking of alcohol that results in significant mental or physical health problems. Part of patient's medical history.
12. Handcap: handicap, part of patient's medical history.
13. SMS_received: frequent reminders of scheduled appointment.
14. No-show: whether the patient made the actual appointment or not. 'Yes/True' means the patient did not make the appointment.

Several questions will be addressed:

1. Factors driving higher attendance frequency? is it age, gender, medical history or enrollment in Bolsa Família program?

2. Does time has an impact on attendance frequency?

3. Are there certain neighborhoods experiencing higher attendance frequency than others? Why?

4. Is appointment scheduling administered properly?
	
Main dependent variable is appointment status 'No-Show’; rest will be the independent ones.


<a id='wrangling'></a>
## Data Wrangling

In [93]:
# load data
master_df = pd.read_csv('../input/noshowappointments/KaggleV2-May-2016.csv')

In [94]:
# inspect
master_df.head()

In [95]:
master_df.info()

In [96]:
master_df.describe()

In [97]:
# Inspect unusual age records that are < 0

master_df[master_df.Age < 0]

In [98]:
# Inspect unusual Handcap records that are > 1

master_df[master_df.Handcap > 1].sample(3, random_state = 42)

In [99]:
# Check for duplicate records

master_df.duplicated().sum()

In [100]:
# Check whether each patient has multiple records

master_df.PatientId.nunique(), master_df.AppointmentID.nunique()

In [101]:
# Inspect multiple records of single patient

master_df[master_df.PatientId.duplicated(keep=False)].sort_values(by='PatientId')

### Data Cleaning

- Convert ID columns to string, no arithmetic operations will be performed on them
- Convert Scheduled and Appointment columns to date time, then calculate elapsed time from scheduling day till appointment day
- Convert No-show to 0 and 1 for convenience
- Make all header lower case and rename columns for better representation
- Drop records with age < 0, assuming that 0 represent new born
- Adjust Handcap values that are > 1 to be 1, assuming that these are just typos
- Unify age variable of patients with multiple records

In [102]:
# Make copy of master df

master_df_clean = master_df.copy()

In [103]:
# Convert ID columns to string, no arithmetic operations will be performed on them

master_df_clean.PatientId, master_df_clean.AppointmentID = master_df_clean.PatientId.astype(str), \
master_df_clean.AppointmentID.astype(str)

In [104]:
# Calculate elapsed time from scheduling day till actual appointment day

# Convert to datetime
master_df_clean.ScheduledDay = master_df_clean.ScheduledDay.apply(np.datetime64)
master_df_clean.AppointmentDay = master_df_clean.AppointmentDay.apply(np.datetime64)

# Extract date only
master_df_clean['scheduledDay_x'] = master_df_clean['ScheduledDay'].dt.date
master_df_clean.scheduledDay_x = master_df_clean.scheduledDay_x.apply(np.datetime64)

# Calculate Elapsed days
master_df_clean['elapsed_days_sch'] = pd.to_timedelta((master_df_clean.AppointmentDay - master_df_clean.scheduledDay_x)).dt.days

# drop new scheduled day column as I need full timestamp data
master_df_clean.drop(columns = ['scheduledDay_x'], inplace =True)

# Inspect
master_df_clean.head()

In [105]:
# Elapsed days distribution
master_df_clean.elapsed_days_sch.plot(kind = 'hist', bins = 20);

In [106]:
# records having -ve values
master_df_clean[master_df_clean.elapsed_days_sch < 0]

In [107]:
# drop -ve values as they are probably erroneous

master_df_clean = master_df_clean[~master_df_clean.elapsed_days_sch < 0].copy()

master_df_clean.reset_index(drop= True, inplace = True)

In [108]:
master_df_clean.shape

In [109]:
# convert No-show to 0 and 1 for convenience

master_df_clean['No-show'].replace(to_replace=['No', 'Yes'], value=[0, 1], inplace=True)

In [110]:
# Make all header lower case and rename columns for better representation

master_df_clean.columns = master_df_clean.columns.str.lower()

master_df_clean.rename(columns={'patientid':'patient_id', 'appointmentid':'appointment_id', 
                                'scheduledday': 'scheduling_day', 'appointmentday': 'appointment_day', 
                                'scholarship':'enrolled_in_Bolsa_Família', 'hipertension': 'hypertension', 
                                'handcap':'handicap', 'no-show':'no_show'}, inplace=True)

In [111]:
# Drop records with age < 0

master_df_clean = master_df_clean[master_df_clean.age >= 0]

master_df_clean.reset_index(drop= True, inplace = True)

In [112]:
# Adjust Handicap values that are > 1 to be 1 as it is not clear whether these entries are errors or representation
# of various stages of Handicap

master_df_clean.loc[master_df_clean.handicap > 1 , 'handicap'] = 1

In [113]:
# Unify age variable of patients with multiple records, I'll use the first recorded age as lead.  
# this will be done through creating a temp sorted dataframe for duplicate patient ids,
# identify gaps in recorded ages for single patient record,
# filter on each unique id then create a dict with IDs being keys and first recorded age as values
# finally map dict keys to main dataframe patient IDs

duplicate_id = master_df_clean[master_df_clean.patient_id.duplicated(keep=False)].sort_values(by='patient_id')

duplicate_id['first'] = duplicate_id['age']

duplicate_id['last'] = duplicate_id['age'].shift(-1)

duplicate_id['gap'] = duplicate_id['age'].shift(-1) - duplicate_id['age']

duplicate_id_fltrd = duplicate_id[(duplicate_id.first != duplicate_id.last) & (duplicate_id.gap == 1)]

duplicate_id_dict = duplicate_id_fltrd.set_index('patient_id').to_dict()['age']

master_df_clean.loc[master_df_clean.patient_id.isin(duplicate_id_fltrd.patient_id), 'age'] = \
master_df_clean.patient_id.map(duplicate_id_dict)

# inspect

master_df_clean[master_df_clean.patient_id.isin(duplicate_id_fltrd.patient_id)].sort_values(by='patient_id', ascending=False)

<li><a href="#toc">Table of contents</a></li>

<a id='eda'></a>
## Exploratory Data Analysis


### Factors driving higher attendance frequency? is it age, gender, medical history or enrollment in Bolsa Família program?

In [114]:
# Grouping multiple records of unique patients into single records and computing average, min, max of some appointment details
# this is to avoid double counting of same patient records while analysing independent variables 
# and also to provide better representation of appointment details to facilitate comparability

# First, group records and make calculations
unique_no_show_df = master_df_clean.groupby('patient_id')[['no_show']].agg(['count', 'mean'])

unique_no_show_df.columns = unique_no_show_df.columns.droplevel()

unique_elapsed_days_sch_df = master_df_clean.groupby('patient_id')[['elapsed_days_sch']].agg(['mean', 'max', 'min'])

unique_elapsed_days_sch_df.columns = unique_elapsed_days_sch_df.columns.droplevel()

unique_sms_received_df = master_df_clean.groupby('patient_id')[['sms_received']].agg(['mean'])

unique_sms_received_df.columns = unique_sms_received_df.columns.droplevel()

dfs = [unique_no_show_df, unique_elapsed_days_sch_df, unique_sms_received_df]

grouped_df = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), dfs) \
.rename(columns = {'count':'appointment_count', 'mean_x': 'no_show_mean', 'mean_y': 'elapsed_days_sch_mean', 
                   'max':'elapsed_days_sch_max', 'min':'elapsed_days_sch_min', 'mean':'sms_received_mean'})

master_df_clean.index = master_df_clean.patient_id

# Then merge to obtain other variables, 
# keep first occuence only as these variables are supposed to be fixed for a single patient

grouped_df = pd.merge(grouped_df, master_df_clean.drop_duplicates(subset=['patient_id'], keep='first'), 
                      left_index=True, right_index=True, how='left')

master_df_clean.reset_index(drop=True, inplace=True)

grouped_df.reset_index(drop=True, inplace=True)

columns_list = grouped_df.columns.tolist()

columns_sort = columns_list[6:18]+columns_list[0:6]

grouped_df = grouped_df[columns_sort]

In [115]:
# duplicate intervals to bypass 'include_lowest' bug as suggested --> (https://github.com/pandas-dev/pandas/issues/23164)
# age groups categorization follows this link,
# https://brasilemsintese.ibge.gov.br/populacao/distribuicao-da-populacao-por-grandes-grupos-de-idade.html
# I split the age group of 15-64 into two groups for better representation

bins = pd.IntervalIndex.from_tuples([(-0.001,0), (0, 14), (14.999,15), (15, 24), (24.999,25), (25, 64), (64.999,65), 
                                     (65,122)])
labels = ['Children','Children','Youth', 'Youth', 'Adults','Adults','Seniors','Seniors']

grouped_df['age_group'] = pd.cut(grouped_df.age, bins, include_lowest =True).map(dict(zip(bins, labels)))

In [116]:
# calculate attendance frequency per each age group as a percentage of total group

age_groups = grouped_df.age_group.value_counts()

low_atnd_freq = grouped_df[grouped_df.no_show_mean > 0].age_group.value_counts()

hgh_atnd_freq = grouped_df[grouped_df.no_show_mean == 0].age_group.value_counts()

out_low = low_atnd_freq / age_groups

out_low.sort_values(ascending=False)

In [117]:
out_hgh = hgh_atnd_freq / age_groups

out_hgh.sort_values(ascending=False)

In [118]:
# Plot the results

fig, axes = plt.subplots(ncols=2, figsize=(8, 4))
fig.tight_layout()
ax1, ax2 = axes

# setup plot labels, box sizes and colors

labels_low = [i + '\n' + str((round(v,2))) for i, v in out_low.items()]
sizes_low = out_low.values
colors_low = [plt.cm.Spectral(i/float(len(labels_low))) for i in range(len(labels_low))]

labels_hgh = [i + '\n' + str((round(v,2))) for i, v in out_hgh.items()]
sizes_hgh = out_hgh.values
colors_hgh = [plt.cm.Spectral(i/float(len(labels_hgh))) for i in range(len(labels_hgh))]

# Draw Plot
squarify.plot(sizes=sizes_low, label=labels_low, color=colors_low, alpha=1, bar_kwargs=dict(linewidth=2, edgecolor="#222222"), text_kwargs=dict(fontsize=11), ax=ax1);

squarify.plot(sizes=sizes_hgh, label=labels_hgh, color=colors_hgh, alpha=1, bar_kwargs=dict(linewidth=2, edgecolor="#222222"), text_kwargs=dict(fontsize=11), ax=ax2);

# Decorate
ax1.set_title('Age group classification of patients' + '\n' + 'with low attendance frequency', fontsize=15, y=1.05, c='orange')
ax2.set_title('Age group classification of patients' + '\n' + 'with high attendance frequency', fontsize=15, y=1.05, c='green')
ax1.axis('off'); ax2.axis('off');

In [119]:
# Create summary table for patient's gender and medical condition 
# to apply value count on the whole dataframe instead of one variable at a time
# Same concept of splitting population as before

summary_high_show_df = grouped_df[grouped_df.no_show_mean == 0].copy()

summary_low_show_df =  grouped_df[grouped_df.no_show_mean > 0].copy()

summary_high_show_df.drop(columns={'patient_id','appointment_id','scheduling_day','appointment_day',
                                   'age','neighbourhood', 'appointment_count', 'no_show_mean',
                                   'elapsed_days_sch_mean', 'elapsed_days_sch_max', 'elapsed_days_sch_min', 
                                   'elapsed_days_sch_min', 'age_group', 'sms_received_mean'}, inplace = True)

summary_low_show_df.drop(columns={'patient_id','appointment_id','scheduling_day','appointment_day',
                                  'age','neighbourhood', 'appointment_count', 'no_show_mean',
                                  'elapsed_days_sch_mean', 'elapsed_days_sch_max', 'elapsed_days_sch_min', 
                                  'elapsed_days_sch_min', 'age_group', 'sms_received_mean'}, inplace = True)

summary_high_show_df = summary_high_show_df.apply(pd.Series.value_counts)

summary_low_show_df = summary_low_show_df.apply(pd.Series.value_counts)

In [120]:
# Calculate relative weight of each condition and plotting

summary_high_show_df.transform(lambda x: round(x / x.sum(),2)).plot(kind='pie', 
                                                                    subplots = True, legend = False, figsize = (20, 10), 
                                                                    fontsize = 15, 
                                                                    autopct = \
                                                                    lambda p: '{:.0f}%'.format(round(p)) if p > 0 else '', 
                                                                    labels = None);

plt.gcf().suptitle('Characteristics of patients attending medical appointments', fontsize=20, y=.68, color = 'brown')

plt.legend(["False", "True", 'Female', 'Male'], bbox_to_anchor=(1.05, 1), loc = 4)

summary_low_show_df.transform(lambda x: round(x / x.sum(),2)).plot(kind='pie', 
                                                                   subplots = True, legend = False, figsize = (20, 10),
                                                                   fontsize = 15, 
                                                                   autopct = \
                                                                   lambda p: '{:.0f}%'.format(round(p)) if p > 0 else '', 
                                                                   labels = None);

plt.gcf().suptitle("Characteristics of patients skipping medical appointments", fontsize=20, y=.68, color = 'brown');

Seniors takes the lead being highest attendance frequency age group, while youth leads the low attendance one. It makes sense as health deteriorates with age.

The dataset is dominated by female presence; it seems that women are receiving extra medical care than men.

Patients' characteristics do not significantly vary between those who attend or skip their medical appointment. However, enrolment in Bolsa Familia program, hypertension and diabetes medical conditions are the highest among patients who attend.

### Does time has an impact on attendance frequency?

In [121]:
# Resample and inspect

grouped_df[grouped_df.no_show_mean > 0].groupby(['scheduling_day'])['no_show_mean'].count().resample('1m').sum()

In [122]:
grouped_df[grouped_df.no_show_mean > 0].groupby(['appointment_day'])['no_show_mean'].count().resample('1m').sum()

In [123]:
grouped_df[grouped_df.no_show_mean == 0].groupby(['scheduling_day'])['no_show_mean'].count().resample('1m').sum()

In [124]:
grouped_df[grouped_df.no_show_mean == 0].groupby(['appointment_day'])['no_show_mean'].count().resample('1m').sum()

In [125]:
# Summary of appointments by weekday

master_df_clean['day'] = master_df_clean.appointment_day.dt.day_name()

master_df_clean.groupby('day')['no_show'].count()

In [126]:
# attendance frequency by weekdays

master_df_clean.groupby('day')['no_show'].value_counts(normalize=True).unstack().plot(
    kind = 'bar', stacked = True, ylabel = 'Frequency', color = ['green', 'orange']);

# Decoration9

plt.title('Attendance frequency by weekday', fontsize = 17, c = 'brown', y = 1.06)

plt.legend(["Attendance", "Absence"], bbox_to_anchor = (1.05, 1), loc = 'upper left')

plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0)

In [127]:
# Check whether decreas in elapsed time between shceduling and actual appointment date impact attendace ferquency
# 16 being average elapsed timed of the dataset

round(grouped_df[(grouped_df.elapsed_days_sch_mean >= 16) & (grouped_df.no_show_mean == 0)].age_group.value_counts(normalize=True),2)

In [128]:
round(grouped_df[(grouped_df.elapsed_days_sch_mean < 16) & (grouped_df.no_show_mean == 0)].age_group.value_counts(normalize=True),2)

May appears to be squeezed compared to other months as most appointments were scheduled to take place during this month. It might be because this period follows the heavy festive months where people are busy celebrating according to [this](https://www.lonelyplanet.com/brazil/narratives/planning/month-by-month), but still this can be considered as an improvement opportunity for appointment administration to spread appointments across months.

There is no preference for specific weekday over the other when it comes to attendance frequency, despite that Thursday/Wednesday experienced improved attendance frequency and Friday/Saturday are the lowest, it’s not significantly different than rest of weekdays. 

No appointments where scheduled to take place on Sunday which is understandable being a weekend day but given that attendance frequency on Saturday, which is also a weekend day, is comparable to the rest of the weekdays it implies that patients are willing to attend appointments on weekends. This can be also considered as an improvement opportunity for appointment administration to spread appointments across weekdays regardless of weekends. 

On the other hand, decrease in elapsed time between scheduling and actual appointment date does not result in a significant decrease in missed appointments. Which is a bit surprising actually and warrants further investigation of underlying appointment scheduling data.

### Are there certain neighborhoods experiencing higher attendance frequency than others? Why?

In [129]:
# Check appointment distribution among Neighborhoods

round(master_df_clean.neighbourhood.value_counts(normalize=True) * 100, 2)

In [130]:
# Create a summary table for attendance frequency per each neighborhood
# to facilitate comparison among each and identify which neighborhood experienced most successful appointments

appointments_by_neighbourhoods = grouped_df.groupby(['neighbourhood'])['appointment_count'].sum()

attendance_by_neighbourhood = grouped_df[grouped_df.no_show_mean == 0]. \
groupby(['neighbourhood'])['appointment_count'].sum()

frequency_of_attendance= attendance_by_neighbourhood / appointments_by_neighbourhoods

frequency_of_attendance = frequency_of_attendance.to_frame().rename(columns={'appointment_count':'frequency_of_attendance'})

appointments_by_neighbourhoods = appointments_by_neighbourhoods.to_frame(). \
rename(columns={'appointment_count':'total_appointments'})

neighbourhoods_summary = pd.merge(frequency_of_attendance, appointments_by_neighbourhoods, 
                                  left_index=True, right_index=True, how='left')

temp_df = master_df_clean.groupby('neighbourhood')[['elapsed_days_sch', 'sms_received']].agg(['mean'])

temp_df.columns = temp_df.columns.droplevel(level=1)

neighbourhoods_summary = pd.merge(neighbourhoods_summary, temp_df, left_index=True, right_index=True, how='left'). \
rename(columns={'elapsed_days_sch':'elapsed_days_sch_mean', 'sms_received':'reminder_frequency'})

neighbourhoods_summary.reset_index(inplace=True)

neighbourhoods_summary.rename(columns={'index':'neighbourhood'}, inplace=True)

neighbourhoods_summary.elapsed_days_sch_mean = neighbourhoods_summary.elapsed_days_sch_mean.astype(int)

neighbourhoods_summary = round(neighbourhoods_summary,2)

In [131]:
# Stats of top 10 neighborhoods with highest attendance frequency

neighbourhoods_summary.sort_values(by='frequency_of_attendance', ascending =False).head(10).describe()

In [132]:
# Stats of top 10 neighborhoods with lowest attendance frequency

neighbourhoods_summary.sort_values(by='frequency_of_attendance').head(10).describe()

In [133]:
# details of top 10 neighborhoods with highest attendance frequency

neighbourhoods_summary.sort_values(by='frequency_of_attendance', ascending =False).head(10)

In [134]:
# details of top 10 neighborhoods with lowest attendance frequency

neighbourhoods_summary.sort_values(by='frequency_of_attendance').head(10)

In [135]:
# will compare age and gender composition among top 2 neighborhood of each highest and lowest groups as 
# other variables are approximatly similar and will be further investigated in appointment adminstration analysis section 

print('ILHA DO BOI gender and age composition :', '\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'ILHA DO BOI'].gender.value_counts(normalize=True),2),'\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'ILHA DO BOI'].age_group.value_counts(normalize=True),2),'\n','\n',
      '-'* 20,'\n','\n',
      'AEROPORTO gender and age composition :', '\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'AEROPORTO'].gender.value_counts(normalize=True),2),'\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'AEROPORTO'].age_group.value_counts(normalize=True),2)
     )

In [136]:
print('ILHA DO FRADE gender and age composition :', '\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'ILHA DO FRADE'].gender.value_counts(normalize=True),2),'\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'ILHA DO FRADE'].age_group.value_counts(normalize=True),2),'\n','\n',
      '-'* 20,'\n','\n',
      'JESUS DE NAZARETH gender and age composition :', '\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'SANTOS DUMONT'].gender.value_counts(normalize=True),2),'\n','\n',
      round(grouped_df[grouped_df.neighbourhood == 'SANTOS DUMONT'].age_group.value_counts(normalize=True),2)
     )

In [137]:
# Check if a neighborhood didn’t have any successful appointment

neighbourhoods_summary[neighbourhoods_summary.frequency_of_attendance.isnull()]

In [138]:
# Check details of this neighborhood

master_df_clean[master_df_clean.neighbourhood == 'ILHAS OCEÂNICAS DE TRINDADE']

Comparing summary statistics of top 10 neighborhoods having highest and lowest attendance frequency indicates that neither shorter elapsed time nor higher frequent reminders have a significant impact on attendance frequency.

By examining the demographic composition of top neighborhoods in from each attendance category (highest/lowest) we can conclude that neighborhoods with demographic composition of less senior patients had less attendance frequency. This confirms my earlier observation that older patients have high attendance frequency.

However, ILHAS OCEÂNICAS DE TRINDADE neighborhood did not experience any successful appointments contrary to my expectation given the composition of gender and age group. Maybe because elapsed time was 28 days and no frequent reminders were sent to patients.

This will be further investigated in appointment administration section.

It warrants additional data to conclude why exactly some Neighborhoods experience higher attendance frequency than others (nature of visits i.e. is it for severe illness 'cancer for example' or just customary, distance between patient and appoint location, commute times, venue, etc...)

### Is appointment scheduling administered properly?

In [139]:
# inspect patients with multiple appointments as it will give better overview on scheduling practices

grouped_df[grouped_df.appointment_count > 1].sort_values(by='appointment_count', ascending=False).head(10)

In [140]:
# inspect records of patients with several appointments

master_df_clean[(master_df_clean.patient_id == '1484143378533.0')].sort_values(by='scheduling_day') \
.drop(columns={'enrolled_in_Bolsa_Família', 'hypertension', 'diabetes', 'alcoholism', 'handicap'})

In [141]:
master_df_clean[(master_df_clean.patient_id == '17798942295934.0')].sort_values(by='appointment_day') \
.drop(columns={'enrolled_in_Bolsa_Família', 'hypertension', 'diabetes', 'alcoholism', 'handicap'})

In [142]:
# Calculate attendace frequency of patients having multiple appointments in the same day

temp_df = master_df_clean[master_df_clean.patient_id.duplicated(keep=False)].sort_values(by='patient_id')

temp_df['gap'] = temp_df['appointment_day'].shift(-1) - temp_df['appointment_day']

temp_df['gap'] = temp_df['gap'].dt.days

temp_df_fltrd = temp_df[(temp_df.no_show == 1) & (temp_df.gap == 0)]

round(temp_df_fltrd.shape[0] / temp_df[temp_df.no_show == 1].shape[0],2)

In [143]:
# Heatmap for appointment stats

appointment_stats = master_df_clean.groupby(['appointment_id'])[['no_show', 'sms_received', 'elapsed_days_sch']].mean()

# Plot

plt.figure(figsize=(10,6))

sns.heatmap(appointment_stats.corr(), xticklabels = appointment_stats.corr().columns, 
            yticklabels = appointment_stats.corr().columns, cmap = 'BuPu', center = 0, annot = True, linewidths = .01)

# Decorations

plt.title('Appointment\'s Stats Heat Map', fontsize = 20, color = 'purple' , y = 1.06)

plt.xticks(fontsize = 13)

plt.yticks(rotation = 0, fontsize = 13, va = "center");

I'll comment on appointment administration activities then will discuss my concerns on the dataset itself

##### Analysis of appointment administration activities

8% of missed appointments were scheduled in the same day, this is noted for patients having multiple appointments. Another development opportunity for appointment administration is to steer away from scheduling multiple appointments in the same day; unless there is a need to.

Friday is a common day for missed appointments as noted from the detailed records inspected above and those of ILHAS OCEÂNICAS DE TRINDADE neighborhood. This is also confirmed by earlier analysis of attendance by weekday as Friday was one of the lowest.

Frequent reminders are associated with the length of elapsed days as noted from +ve correlation between sms received and elapsed time. However, correlation coefficient indicate weak linear relationship which can be observed in the appointment records of the patients above. Take patient # 1484143378533 as an example, we can observe that reminders are not sent systematically for similar elapsed time period same as for patient # 17798942295934 but with less frequency. Switch sorting of each table to reproduce these observation. 

This can be an area for improvement in appointment administration by following a systematic approach in sending reminders to patients and/or increasing the frequency or methods of reminders. However, given the weak correlation that is almost nonlinear between the frequency of reminders and attendance; associated costs of increasing or changing reminder methods need to be closely examined. 

Weak +ve correlation that is almost nonlinear between elapsed time and attendance frequency need to be investigated further, as a strong +ve linear relationship is expected unless there are limited sources of medical services providers.


##### concerns on data integrity

Actually, appointment administration and related data needs a lot further investigation than any other independent variable in this dataset to confirm data integrity, as several red flags were noted as follows:

1. Insignificant impact of shorter appointment lead times on attendance frequency
2. Time stamps and appointment ID discrepancies, discussed further below.

Upon inspecting the appointment record of the 2 patients above, having around 80% attendance frequency and a total of 50 appointments, I noticed that:

1. Several appointments were scheduled in the same day within a very short time, sometimes less than a minute!

2. Appointment ID sequence gap is significant compared to the short scheduling time.

To elaborate more, let’s take the record of patient # 1484143378533 as an example. We can see that there are 5 appointments scheduled on the 2nd of May’16 with the following time stamps sorted in an ascending order ‘18:56:40’, ‘18:57:31, ‘18:59:07’, ‘19:00:08’,’ 19:01:32’. Each appoint has a unique ID as follows ‘5649276’, ‘5649283’,’ 5649297’,’ 5649306,’ 5649316’. Such data sparks a lot of concerns:

- How is it possible to have such quick calls/registrations?

- How come gaps in appointment ID are significant within such short time span? Are all Neighborhoods linked to the same registration system?

- What’s the point of having such pattern of registration, does they all relate to each other? Should some of them be cancelled?

- Is No show record for these particular appointments correct?

- IF it’s not a registration problem/error rather a data gathering one, does this affect the accuracy of other independent variables?

Unfortunately, answers to these questions can’t be extracted from the current dataset. But I’m Curious!

<a id='conclusions'></a>

## Conclusion

1. Senior age group have higher attendance frequency than others
	
2. Youth age group most likely will skip their medical appointments
	
3. Enrolment in Bolsa Familia program, hypertension and diabetes medical conditions are common characteristics of patients with higher attendance frequency
	
4. Seniors and adults age group shows a slight improvement in attendance frequency with shorter lead times.

5. No preference for specific weekday over the other when it comes to attendance frequency.


## Appointment administration improvement opportunities 

1. Spread appointments across months and weekdays regardless of weekends.

2. Avoid scheduling multiple appointments in the same day for a single patient. Unless there is a need to.

3. Follow a systematic approach when sending reminders to patients and consider increasing the frequency and/or methods of reminders while closely monitoring the associated costs of doing so.
	

## Limitations
	
- How was the appointment scheduled is not clear.
	
- Sufficient details about appointments are not provided (Type, employee ID, physician, venue, etc…).
	
- Medical history of children has some unusual entries like alcoholism. I did not disregard this.
	
- Count and timing of sms reminders are not provided
	
- I assumed age of 0 relates to newborns and not data entry error

- Appointment administration and related data anomalies can lead to different interpretations if proved to be valid, actually it casts doubts on the integrity of the whole dataset.

<li><a href="#toc">Table of contents</a></li>

<a id='predictions'></a>

## Appointment predictions

#### modeling workflow:

- Data preprocessing including:
  - Removing data points having identical features but different class labels, since I can't conclude on the reason for such instances from the given data set.
  - Outlier detection and removal using PyOD.
- Checking for multicollinearity.
- Feature selection using different techniques.
- Testing several models with different feature sets.
- Hyperparameter tuning for best performing model(s).
- Testing multiple sampling techniques to address class imbalances.
- Concluding on best performing model and features.

check out this [GOLD MINE](https://machinelearningmastery.com/), the blog has been a great help guiding this modeling.

In [144]:
master_df_clean_ml = master_df_clean.copy()

master_df_clean_ml.age = master_df_clean_ml.age.astype('int64')

master_df_clean_ml.no_show = master_df_clean_ml.no_show.replace({0:1, 1:0})

master_df_clean_ml.rename(columns = {'no_show': 'attend'}, inplace = True)

gender = pd.get_dummies(master_df_clean_ml.gender, drop_first=True)

master_df_clean_ml = pd.concat([master_df_clean_ml, gender], axis = 1)

master_df_clean_ml.drop(columns=['appointment_id', 'gender', 'neighbourhood', 
                                 'scheduling_day', 'day'], axis =1 , inplace = True)

master_df_clean_ml.head()

In [145]:
master_df_clean_ml.drop(columns = ['patient_id', 'appointment_day'], inplace = True)

columns_list = master_df_clean_ml.columns.tolist()

columns_sort = columns_list[0:7]+columns_list[8:]+columns_list[7:8]

master_df_clean_ml = master_df_clean_ml[columns_sort]

master_df_clean_ml.columns

In [146]:
master_df_clean_ml.shape

In [147]:
# removing identical features having different classes, since I cannot conclude why such instance happen from the available
# data. Such instances will degrade model performance

# Mask for identical feats with different class labels

dup = master_df_clean_ml[(master_df_clean_ml.duplicated(subset = master_df_clean_ml.columns[0:9], keep = False))].\
sort_values(by = ['age', 'enrolled_in_Bolsa_Família', 'hypertension', 'diabetes',
       'alcoholism', 'handicap', 'sms_received', 'elapsed_days_sch', 'M'], ascending =True)

# Creating a matrix to capture rows to be deleted, all duplicated rows with different class labels will 
# have a unique pattern, it's better to inspect the data outside of Jupyter Notebook to better identify the pattern.

dup['unique'] = dup['attend'].shift(-1) - dup['attend']

dup['next'] = dup['elapsed_days_sch'].shift(-1) - dup['elapsed_days_sch']

dup['previous'] = dup['elapsed_days_sch'].shift() - dup['elapsed_days_sch']

dup['prev_unique'] = dup['attend'].shift() - dup['attend']

dup.to_csv('dup.csv')

dup.tail(10)

Indexes 24127 and 74219 are an example of duplicated exact features having different class labels. Here we have 3 instances, index 86776 as well, of which 2 are having different class labels; I'll be removing instances similar to these 2 from the data as I can't conclude on the exact reason for such difference and it will degrade model performance.

In [284]:
# Capturing all rows to be deleted

dup = dup[(((dup.attend == 0) & (dup.unique == 1) & (dup.next == 0) & (dup.previous == 0) & (dup.prev_unique == 1)) |
           ((dup.attend == 1) & (dup.unique == -1) & (dup.next == 0)) |
           ((dup.attend == 0) & (dup.previous == 0) & (dup.prev_unique == 1)) | 
           ((dup.attend == 1) & (dup.unique == -1) & (dup.previous == 0)) |
           ((dup.attend == 0) & (dup.unique == 1) & (dup.next == 0)) |
           ((dup.attend == 0) & (dup.unique == 0) & (dup.next == 0) & (dup.previous == 0) & (dup.prev_unique == 1)) |
           ((dup.attend == 0) & (dup.unique == 0) & (dup.next == 0) & (dup.previous == 0) & (dup.prev_unique == 0)) |
           ((dup.attend == 0) & (dup.unique == 1) & (dup.next == 0) & (dup.previous == 0) & (dup.prev_unique == 0)))]

# Removing captured rows

master_df_clean_ml = master_df_clean_ml[~master_df_clean_ml.index.isin(dup.index)]

# Separate features and labels

features = master_df_clean_ml.drop(columns = ['attend'])
labels = master_df_clean_ml['attend']

# Rescaling

features_resc = StandardScaler().fit_transform(features)

In [285]:
dup.shape, master_df_clean_ml.shape, features.shape

In [150]:
# Outlier detection and removal, setting n_component to 1 as I want the analysis to be based on the PC explaining the 
# highest variance in the dataset

pca_pyod = PCA_pyod(n_components = 1).fit(features_resc)

features_pyod = features.copy()

# identifying instances labeled as outliers for further analysis

features_pyod['outlier_score'] = pca_pyod.decision_scores_

features_pyod['outlier_labels'] = pca_pyod.labels_

# analysing inliers
pyod_inliers = features_pyod[features_pyod.outlier_labels == 0]

pyod_inliers.describe()

In [151]:
features.shape, pyod_inliers.shape

In [152]:
features.alcoholism.value_counts()[1], features.handicap.value_counts()[1]

A total of 8,201 instances were identified as outliers, of which 5,093 related to alcoholism and handicap which were removed completely. I'll agree with such cleanup since these two features did not have material effect on attendance frequency as noted in my earlier EDA.

I'll explore further what constitute the rest of the outliers identified apart from alcoholism and handicap features.

In [153]:
# analysing outliers

pyod_outliers = features_pyod[features_pyod.outlier_labels == 1]

pyod_outliers.sort_values(by = 'outlier_score').tail()

In [154]:
# inspect records related to the ones having highest outlier scores

pyod_outliers[(pyod_outliers.age == 73)].sort_values(by = 'elapsed_days_sch', ascending=False)

In [155]:
features[features.age == 73].plot(kind='hist', y = 'elapsed_days_sch', bins = 30);

Cleanup is done basically on instances having abnormal elapsed days along with being handicapped/alcoholism.

In [156]:
features_pyod = features_pyod[features_pyod.outlier_labels == 0]

labels_pyod = labels[labels.index.isin(features_pyod.index)]

features_pyod.drop(columns=['outlier_score', 'outlier_labels'], inplace = True)

features_pyod.describe()

In [157]:
# Dropping unnecessary columns after outlier removal

features_pyod.drop(columns=['alcoholism', 'handicap'], inplace = True)

# Rescaling

features_resc_pyod = StandardScaler().fit_transform(features_pyod)

In [158]:
features_pyod.shape, features_resc_pyod.shape, labels_pyod.shape

In [159]:
# Checking for severe multicollinearity 

# In regression, "multicollinearity" refers to predictors that are correlated with other predictors (variable/feature).  
# Multicollinearity occurs when the model includes multiple factors that are correlated not just to the response variable,
# but also to each other.

# A VIF of 1 (the minimum possible VIF) means the tested predictor is not correlated with the other predictors. 
# The rule of thumb is that VIF shouldn’t be higher than 10.

vif_data = pd.DataFrame()
vif_data['features'] = features.columns
vif_data['vif'] = [variance_inflation_factor(features.values, i) for i in range(len(features.columns))]

vif_data

In [160]:
# Feature selection chi2

# chi-square test measures dependence between stochastic variables (whose values depend on outcomes of a random phenomenon), 
# so using this function “weeds out” the features that are, the most likely to be independent of class and therefore 
# irrelevant for classification.

features_best = SelectKBest(chi2, k = 3)
features_best.fit_transform(features, labels)
cols = features_best.get_support(indices=True)
features_best_df_chi2 = features.iloc[:,cols]


# # printing scores, it’s better to visualize though
# for i in range(len(features_best.scores_)):
#     print('%s: %f' % (features.columns[i], features_best.scores_[i]))
    
# plot the scores
plt.bar([features.columns[i] for i in range(len(features_best.scores_))], features_best.scores_, );
plt.xticks(rotation = 90)
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

In [161]:
# Feature selection f_classif

features_best = SelectKBest(k = 3)
features_best.fit_transform(features, labels)
cols = features_best.get_support(indices=True)
features_best_df_fcla = features.iloc[:,cols]

# # for i in range(len(features_best.scores_)):
# #     print('%s: %f' % (features.columns[i], features_best.scores_[i]))
    
# plot the scores
plt.bar([features.columns[i] for i in range(len(features_best.scores_))], features_best.scores_);
plt.xticks(rotation = 90)
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

In [162]:
# Feature selection Mutual

features_best = SelectKBest(mutual_info_classif, k = 3)
features_best.fit_transform(features, labels)
cols = features_best.get_support(indices=True)
features_best_df_mutual = features.iloc[:,cols]

# for i in range(len(features_best.scores_)):
#     print('%s: %f' % (features.columns[i], features_best.scores_[i]))

# plot the scores
plt.bar([features.columns[i] for i in range(len(features_best.scores_))], features_best.scores_);
plt.xticks(rotation = 90)
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

In [286]:
# Feature selection Lasso

lasso = LassoCV().fit(features_resc, labels)
importance = np.abs(lasso.coef_)
feature_names = np.array(features.columns)

plt.bar(height=importance, x=feature_names)
plt.title("Feature importances via coefficients")
plt.xticks(rotation = 90)
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

As noted earlier in the EDA section, current features are not informative of output class.

In [287]:
# Select only top 3 features from lasso

threshold = np.sort(importance)[-3]

tic = time()

sfm = SelectFromModel(lasso, threshold=threshold).fit(features_resc, labels)

toc = time()

print(f"Done in {toc - tic:.3f}s", '\n' ,"Features selected by SelectFromModel: "
      f"{feature_names[sfm.get_support()]}")

features_best_df_sfm = features[feature_names[sfm.get_support()]]

In [288]:
# Building a dataframe for all unique best selected features across all methods

cols = [features_best_df_chi2.columns, features_best_df_fcla, features_best_df_mutual.columns, features_best_df_sfm]

feats = {x for l in cols for x in l}

features_top = features[feats]

features_top

In [289]:
# Dataframe of top common feature(s) across all methods

features_ulti = features_top[['elapsed_days_sch']]

In [290]:
features.shape, features_pyod.shape

In [291]:
labels.value_counts(normalize=True)

I'll evaluate models using repeated stratified k-fold cross-validation, using different feature sets.

The k-fold cross-validation procedure provides a good general estimate of model performance that is not too optimistically biased, compared to a single train-test split. k=10 means each fold will contain about x/10 examples. Where x represents n_samples. So using the features set, each fold will have 8,204 examples.

Stratified means that each fold will contain the same mixture of examples by class, that is about 92% to 8% attendance and no-show cases respectively if we are using the features set.

Repeated means that the evaluation process will be performed multiple times to help avoid fluke results and better capture the variance of the chosen model. I'll use 3 repeats. This means a single model will be fit and evaluated 10 * 3 or 30 times and the mean and standard deviation of these runs will be reported.

In [292]:
# All models

models = [('DC', DummyClassifier(strategy= 'stratified')), ('LR', LogisticRegression(class_weight='balanced')), 
          ('NB', GaussianNB()), ('DT', tree.DecisionTreeClassifier(class_weight='balanced'))]

feats = [('features', features), ('features_rescaled', features_resc), ('Chi2', features_best_df_chi2), 
         ('f_classif', features_best_df_fcla), ('mutual_calssif', features_best_df_mutual), 
         ('lasso', features_best_df_sfm), ('top_features', features_top), ('ultimate_features', features_ulti)]

results = []

names = []

metric = 'roc_auc'

for name, model in models:
    for feat_name, feat in feats:
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
        scores = cross_val_score(model, feat, labels, scoring=metric, cv=cv, n_jobs=-1)
        names.append(name)
        results.append(scores)
#         print('%s : \n %s ---> Mean ROC_AUC ---> %.2f (Std: %f)' % (name, feat_name, scores.mean(), scores.std()))

plt.figure(figsize=(10,5))
plt.boxplot(results, labels=names, showmeans=True);
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

DT - Decision Tree is highest performing model using the 6th feature set which is selected using Lasso.

In [293]:
feats = [('features_pyod', features_pyod), ('features_rescaled_pyod', features_resc_pyod)]

results = []

names = []

for name, model in models:
    for feat_name, feat in feats:
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
        scores = cross_val_score(model, feat, labels_pyod, scoring=metric, cv=cv, n_jobs=-1)
        names.append(name)
        results.append(scores)
#         print('%s : \n %s ---> Mean ROC_AUC ---> %.2f (Std: %f)' % (name, feat_name, scores.mean(), scores.std()))

plt.figure(figsize=(10,5))
plt.boxplot(results, labels=names, showmeans=True);
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

LR - Logistic Regression is highest performing model using features trimmed by PyOD. I'll be comparing between these two models.

### Precision: 
Is the ratio of correctly predicted positive observations (TP) to the total predicted positive observations (TP/TP+FP). Taking attendance calss as an example, precision address the following question: of all patients that the classifier labeled as attended, how many actually attended? in other terms we can say that our model predicts X% of the time, a patient will attend.

### Recall (Sensitivity): 
Is the ratio of correctly predicted positive observations to all observations related to the class in question. Taking attendance class as an example, Recall address the following question: of all the patients that truly attended, how many did the classifier labeled? (TP/TP+FN)

### F1 score: 
Is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. F1 is usually more useful than accuracy, especially in an uneven class distribution. Accuracy works best if false positives and false negatives have similar values. If the value of false positives and false negatives are very different, it’s better to look at both Precision and Recall. (2*(Recall * Precision) / (Recall + Precision))

### Confusion Matrix:

Confusion matrix does a pretty good job clearing any 'confusion' that the above terms might have caused to you. The below table shows how the results of predictions vs actuals are displayed in scikit-learn's confusion matrix, we can shuffle how results are displayed using the 'label' parameter. Note that not all confusion matrixes do display results in same setup, so know your matrix before judging a classifier.

Columns represent predictions, rows are actuals.

                   Predictions
    ---------------------------------------
    |        No         |        Yes      |
    |------------------ |-----------------|
    | 'True Negative'   | 'False Positive'|
    | 'False Negative'  | 'True Positive' |
    

Let's take first confusion matrix we have below (logistic regression cross validation) as an example and link the matrix to the metric discussed above and shown in the classification report. For 'not_attend' class label:

- Precision: (TP/TP+FP) = (3619 / 3619 + 14437) = .20
- Recall: (TP/TP+FN) = (3619 / 3619 + 1131) or (3619 / 4750*) = .76
- F1-score: (2*(Recall * Precision) / (Recall + Precision)) = (2*(.20 * .76) / (.20 + .76)) = .316 (.32 rounded)

(*) can be found in 'support' column. The column 'support' displays how many object of class x were in the test set, it is not a representation of the total number of samples for this specific class unless cross validation is used.

Given the current setup, 'not_attend' class is displayed as true negative/false negative in the matrix, how does this fits into the precision/recall formula? Positive and negative in this case are generic names for the predicted classes, you can invert what you are "trying to predict" and thereby get two scores as displayed in the classification report; actually the classification report auto calculate these metrics regardless of your prediction goals.

### ROC_AUC:

I'll use ROC_AUC as a general performance measure across different models, however, final decision on which model performs best is not dependent on the ROC_AUC threshold.

I'd consider a classifier to be working properly if it minimize False Positives (patients predicted to show-up but actually did not) even at the expense of high False Negatives (patients predicted to NOT show-up but actually did show-up), below is a clarification for such decision:

- Normally, all patients are expected to show-up to their medical appointments.
- Appointment setting teams need to focus efforts and resources on those who have a higher tendency of actually not showing up.

So if a classifier did predict a show-up, which is normally expected, but it turns out to be a no show; then we've excluded this case (patient) from team's scope for setting up a thorough appointment reminder process. This is not desirable.

Thus, I'll be giving priority for recall on the minority classes (no-shows) while minimizing false positives.

More on ROC_AUC [here](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

<a id='logcross'></a>

In [294]:
clf = LogisticRegression(solver='liblinear', class_weight='balanced')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(clf, features_pyod, labels_pyod, scoring=metric, cv=cv, n_jobs=-1)
predictions = cross_val_predict(clf, features_pyod, labels_pyod, cv = 10, n_jobs = -1)

print(classification_report(labels_pyod, predictions, target_names=['not_attend', 'attend']), '-'*40, '\n',
      confusion_matrix(labels_pyod, predictions), '\n', '-'*40, '\n',
      'Mean ROC_AUC: %f (Std: %f)' % (scores.mean(), scores.std()))

<li><a href="#treecross">DT cross_val</a></li>

<a id='logroc'></a>

In [295]:
labels_pyod.value_counts(normalize=True)

In [296]:
features_train, features_test, labels_train, labels_test = train_test_split(features_pyod, labels_pyod, 
                                                                            test_size=0.2, random_state=42)

clf = LogisticRegression(C = 10, class_weight = 'balanced', max_iter = 300)

t0 = time()
clf.fit(features_train, labels_train)
print("training time:", round(time()-t0, 3), "s")

t0 = time()
pred = clf.predict(features_test)
print("prediction time:", round(time()-t0, 3), "s")

print(classification_report(labels_test, pred, target_names=['not_attend', 'attend']), '-'*40, '\n',
      confusion_matrix(labels_test, pred), '\n', '-'*40)

fpr, tpr, threshold = roc_curve(labels_test, pred)

roc_auc = auc(fpr, tpr)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate');

<li><a href="#treeroc">DT ROC_AUC</a></li>

<a id='treecross'></a>

In [297]:
clf = tree.DecisionTreeClassifier(class_weight='balanced')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(clf, features_best_df_sfm, labels, scoring=metric, cv=cv, n_jobs=-1)
predictions = cross_val_predict(clf, features_best_df_sfm, labels, cv = 10, n_jobs = -1)

print(classification_report(labels, predictions, target_names=['not_attend', 'attend']), '-'*40, '\n',
      confusion_matrix(labels, predictions), '\n', '-'*40, '\n',
      'Mean ROC_AUC: %f (Std: %f)' % (scores.mean(), scores.std()))

In [298]:
master_df_clean_ml.shape, features_best_df_sfm.shape

In [299]:
# Current status of SMS reminders distribution

master_df_clean_ml.groupby(['sms_received', 'attend']).size()

If we look at SMS reminder distribution that is currently prevailing before applying the model, we can note that a total of 23,269 SMS were sent to patients of which 13% (2,921) did not attend, while 87% (20,348) of reminders were sent to patient who actually attended.

If our model was deployed the focus would shift to patients who are predicted to not attend with a total of 28,033 ('No' predictions). Thus, SMS reminders would have targeted 20% (5,554) of patients who did not actually attend their appointments while maintaining even lower reminder rate of around 80% (22,479) for patients who did.

Moreover, this 20% (5,554) targeted by our model do represent 88% of all patients who actually did not attend their appointment (6,300) vs. only 46% before deploying the model.

<li><a href="#logcross">LR cross_val</a></li>

<a id='treeroc'></a>

In [300]:
features_train, features_test, labels_train, labels_test = train_test_split(features_best_df_sfm, labels, 
                                                                            test_size=0.2, random_state=42)

clf = tree.DecisionTreeClassifier(class_weight='balanced', criterion='entropy', max_depth=100, random_state=42)

t0 = time()
clf.fit(features_train, labels_train)
print("training time:", round(time()-t0, 3), "s")

t0 = time()
pred = clf.predict(features_test)
print("prediction time:", round(time()-t0, 3), "s")

print(classification_report(labels_test, pred, target_names=['not_attend', 'attend']), '-'*40, '\n',
      confusion_matrix(labels_test, pred), '\n', '-'*40)

fpr, tpr, threshold = roc_curve(labels_test, pred)

roc_auc = auc(fpr, tpr)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate');

<li><a href="#logroc">LR ROC_AUC</a></li>

In [301]:
clf = tree.DecisionTreeClassifier()

over = SMOTE(random_state=42)

feat_resampled, lab_resampled = over.fit_resample(features_train, labels_train)

t0 = time()
clf.fit(feat_resampled, lab_resampled)
print("training time:", round(time()-t0, 3), "s")

t0 = time()
pred = clf.predict(features_test)
print("prediction time:", round(time()-t0, 3), "s")

print(classification_report(labels_test, pred, target_names=['not_attend', 'attend']), '-'*40, '\n',
      confusion_matrix(labels_test, pred), '\n', '-'*40)

fpr, tpr, threshold = roc_curve(labels_test, pred)
roc_auc = auc(fpr, tpr)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate');

In [302]:
under = RandomUnderSampler(random_state=42)

feat_resampled, lab_resampled = under.fit_resample(features_train, labels_train)

t0 = time()
clf.fit(feat_resampled, lab_resampled)
print("training time:", round(time()-t0, 3), "s")

t0 = time()
pred = clf.predict(features_test)
print("prediction time:", round(time()-t0, 3), "s")

print(classification_report(labels_test, pred, target_names=['not_attend', 'attend']), '-'*40, '\n',
      confusion_matrix(labels_test, pred), '\n', '-'*40)

fpr, tpr, threshold = roc_curve(labels_test, pred)
roc_auc = auc(fpr, tpr)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate');

Different sampling techniques did not provide material difference in model performance.

<a id='predconc'></a>

### Predictions conclusion -  Who's To Blame?

Well, since I have a lot of unanswered questions highlighted in the conclusion section of the EDA and the fact that available features are not informative; I can't really decide who is to blame.

As noted earlier while analysing the records of patients having multiple appointments, where several discrepancies were noted in appointment setting and follow-up via SMS reminders, there is clearly a room for improvement in appointment administration procedures.

Decision Tree classifier has the best performance minimizing false positives while maintaining high recall and AUC_ROC score over minority class (patients who do not attend). 

While this model might not be pretty given high false negative rate, it does a decent job channeling efforts towards patients who are most likely to miss their medical appointments.

<li><a href="#treecross">DT cross_val</a></li>

<li><a href="#toc">Table of contents</a></li>

<a id='analysis'></a>

## In-depth analysis and Feature Engineering

Towards the end of EDA section we've inspected the records of multiple patients and noted some discrepancies in the timestamps of scheduling dates, I believe that this warrants deeper analysis as it can provide additional insights on reasons for 'no show' that may lead to better prediction results.

We will first split the dataset into patients who have several records of medical appointments, and those who don't. Then we will analyse a selected sample of patients with multiple appointment records and identify patterns that may be contributing to an appointment being a 'no show'. Finally we’ll engineer some features that capture these patterns and test whether they can be exploited by a classifier to improve predictions. This is actually a challenging task mainly because of the following:

- Absence of sufficient details for each appointment, some of these missing details would greatly enhance our understanding of the underlying cause of 'no show' such as:
    - Type of appointment (initial, follow-up or substitute) 
    - Medical specialties (Dermatology, Diagnostic radiology, Ophthalmology, etc.), 
    - Venue/location (medical service provider), 
    - Time of appointment (hour of the day), 
    - How is scheduling done (walk-in, call, online, etc.). 
- Class imbalance and overlap issue

Given the current data we are basically analyzing patients' 'no show' behavior based mainly on features related to timing of appointments, neighborhood and some basic knowledge of patients' characteristics; while in fact a lot of other factors/features can actually contribute to 'no show' behavior that have much more discriminatory power than what is already available.

Another complication is the imbalanced nature of the dataset, which means that +ve examples 'no show' are underrepresented and this pose a challenge as predictions tends to be skewed towards the dominant class (-ve 'show')

One important and very critical component of a machine learning project is organizing the workflow, below is an example of a generalized workflow structure:

- Problem definition
- Exploratory data analysis, this must include an in-depth look at samples of data not just plots and summaries
- Preprocessing
- Selecting an evaluation metric
- Establishing a baseline
- Modeling and evaluation

Whatever the structure followed there are some key concepts that need to be considered during preprocessing of data for modeling:

- Split the data into train/test sets early on
- Perform preprocessing on both sets separately, or just ensure that training data is independent of testing/validation data
- Cross validate

Note: I'll be highlighting problematic preprocessing steps along the way and demonstrate their effect on prediction results, if any.

So, based on EDA section we can redefine the problem statement for this dataset as *'imbalanced binary calcification problem having class overlapping issue and poor discriminatory features'*

Our selected evaluation metrics will be *F measure* and *Precision Recall Area Under the Curve (PR AUC)* as these two metrics are best suited for the imbalanced nature of the dataset as we will see later on.

In [527]:
# Copy for fresh start
ml_df = master_df_clean.sort_values(['patient_id', 'appointment_day', 'scheduling_day']).drop('day', axis = 1).copy()

# Convert categorical variables to numeric
ml_df['M'] = pd.get_dummies(ml_df.gender, drop_first=True)

ml_df.head(2)

In [528]:
# Class distribution
ml_df.no_show.value_counts(normalize=True).round(2)

In [529]:
# Split into two datasets one for patients having several appointment records and another for ones having single appointment

df_first = ml_df[~ml_df.patient_id.duplicated(keep = False)].copy()
df_hist = ml_df[ml_df.patient_id.duplicated(keep = False)].copy()

# Sample of patient having multiple appointments
df_hist[df_hist.patient_id == '95294158822869.0']

Several appointments seem to be identical/duplicate, check appointment_id # 5598835, 5598836, 5675614 and 5675615. 
Will remove such appointments but keep +ve samples 'no show' only; having samples with almost identical features but different class labels does affect prediction results negatively such samples are referred to as 'Tomek Links'.

In [530]:
# Dropping duplicate/identical appointments
# We will Keep +ve samples of duplicated appointments that have different class label and
# Only one of the other duplicated samples

# Selecting columns to identify duplicates
mask = df_hist[['patient_id','gender','scheduling_day','appointment_day','age', 'neighbourhood','no_show']].copy() 

# Mark all +ve samples
mask['flag'] = mask.no_show.apply(lambda x: 1 if x ==1 else 0)

# Total +ve samples per group, used to capture identical appointments with different class labels
mask['flag_sum'] = mask.groupby(['patient_id', 'scheduling_day']).flag.transform('sum')

In [531]:
# Sample of patient records
mask[mask.patient_id == '95294158822869.0']

Above are sample of a patient records that are duplicated based on scheduling dates, I'm ignoring other features when filtering duplicates and considering these samples to be erroneous. 

'flag' captures +ve samples and 'flag_sum' captures sum of +ve samples per grouped scheduling day, so if a patient had two appointments with different labels in a single scheduling slot (example index 50411 and 50415) the flag sum would be equal to 1 and this will make it easy to spot and drop the -ve sample.

In [532]:
# -ve samples of duplicated appointments that have different class labels
# flag_sum >= 1 captures +ve samples per group and flag = 0 captures the -ve samples for which a +ve sample exists per the 
# same group
idx_1 = mask[(mask.flag_sum >=1) & (mask.flag ==0)].index

# duplicate observations to be dropped
idx_2 = mask[mask.duplicated()].index

# combined List of indices to be dropped
com_idx = idx_1.append(idx_2)

# applying filter
df_hist = df_hist[~df_hist.index.isin(com_idx)]

# Move single appointments to the other dataframe, as some patients had only two observations of which one was dropped 
# (the -ve one) so it is now considered a single appointment, we will use all these samples later on.
_ = df_hist[~df_hist.patient_id.duplicated(keep=False)]

df_first.append(_)

# Final filtered dataframe
df_hist = df_hist[df_hist.patient_id.duplicated(keep = False)].copy()

# reset index
df_hist.reset_index(inplace=True, drop=True)
df_first.reset_index(inplace=True, drop=True)

# check filter
df_hist[df_hist.patient_id == '95294158822869.0']

Filter is applied correctly as only +ve sample (index # 66815 above vs 50415 in mask) along with one sample per each group is shown in his records. Index has been reset so don't be confused by different index reference.

In [533]:
# class distribution of new dataframe
df_hist.no_show.value_counts(normalize=True).round(2)

Class distribution is slightly different than main dataframe.

<a id='panalysis'></a>

### Analysing patient behavior

We now look at historical records of some patients and try to identify patterns contributing to 'no show' behavior then capture these patterns in new features.

We’ve given care to select samples of patients with different characteristics (age, medical condition, neighborhood, etc.) and at the same time highlight problems previously discussed being mainly poor discriminatory features and conflicting patient behavior.

In [534]:
df_hist[df_hist.patient_id == '22714516796615.0']

First thing to note is that current features are not informative, we can see that almost all features are somewhat similar for appointments having different class labels so they do not provide solid explanation for variation in patient behavior.

This patient had 5 appointments, only first and last one was missed. We can infer from his behavior the following:

- 1st appointment was missed and by looking up the scheduling and appointment date, the day of the week was Thursday and Monday respectively.
- On that same day (first appointment day) he had scheduled another appointment (the 2nd appointment), we may think of it as a substitute appointment for the previous missed one and he attended it. Note that day of the week for this appointment scheduling and appointment date was Monday and Wednesday respectively.
- We may think of 3rd appointment as either:
    - An independent appointment
    - A follow up, given time difference (only 1 day) between previous appointment and scheduling date.
- 4th appointment can be considered as a follow-up for the third appointment considering that it is scheduled on the same day as the previous appointment, and that previous appointment was attended
- 5th and last appointment can be thought of similarly to the 3rd appointment which means that there are different interpretations of what we might think of what that appointment was; what is important to note is that this appointment was also a ‘no show’ and day of the week for both scheduling and appointment date was Thursday and Monday; same as with first appointment that was also a ‘no show’! Even scheduling time of both appointments are somewhat similar.  

So to summarize this subset of data, the patient had total of 5 appointments, attended 3 and missed 2. What's common between these two missed appointments is that scheduling day of week was on Thursday and appointment day of week was on Monday, scheduling time of both appointments also appears to be similar regardless of the scheduling day. 

While such observations could be confined only to this particular patient and does not repeat for others (in terms of specific dates and time), what is obvious is that timing may have an impact on ‘no show’ behavior; we've seen this earlier in the last section of EDA when we looked at records of some patients.

In [535]:
df_hist[df_hist.patient_id == '11163631268489.0']

- There are a total of 2 missed appointments for this patient, each were scheduled on the same day of previously attended appointment so we will consider them as a follow-up appointments as follows:

    - app_id # 5690657 & 5690667 scheduled on same day of app_id 5669705, lets refer to them as c_1
    - app_id # 5719037 & 5719053 scheduled on same day of app_id 5690657, lets refer to them as c_2

- c_1’s first follow-up appointment (app # 5690657) was attended and the 2nd (app # 5690667) was not. Possible explanation is that the 2nd is separated by 1 day only from the 1st (assuming they both relate) so the patient was reluctant to go to that appointment or it may be a double booking.

- c_2’s first follow-up (app 5719037) was not attended and the 2nd (app 5719053) was. Possible explanation is that the 2nd was scheduled on a day having two appointments and the patient preferred to attend both on a single day instead of multiple visits in a short time span (also, assuming a relationship between both appointments exists)

We can create couple of features to capture differences in days between current and previous appointment and to mark multiple follow-up appointments that are scheduled on different days.

Anyway, we can still note the effect of time on 'no show' behavior, one of the common criteria between this patient and the previous one is the 'no show' on scheduling day '2016-05-12' other common criteria is that both are of the same age. 

Elapsed days between scheduling and appointment days don’t seem to affect this patient's 'no show' behavior much as in fact he did attend couple of appointments with relatively high gap between both dates (last two appointments).

Additionally, both c_1 and c_2 appointments are scheduled in a very short time (almost one minute between each) and its not clear whether one should cancel the other or they are really independent from one another but given the difference in appointment dates we are going to consider each as an independent appointment.

In [536]:
df_hist[df_hist.patient_id == '1122443646527.0']

3 appointments were not attended of which two were scheduled in a very short time span of less than a minute difference, this is now a repeating pattern as we noted the same for the previous patient.

In [537]:
df_hist[df_hist.patient_id == '114656291287.0']

This patient has 0 absence frequency and his appointments include almost all examples from the patterns noted in previous patients. This makes sense as up till now we are looking at records of patients individually and it is expected that one patient behavior be different from another.

In [538]:
df_hist[df_hist.patient_id == '112397157856688.0']

Child (age 9) only one missed appointment (app # 5685266) for which a substitute was created (app # 5715126), possible 'no show' reason is absence of reminders 'sms received'.

Conclusion:

We saw that there is no unified pattern of 'no show' among patients; rather each subset of patients has its own behavior. This is understandable given the nature of the problem and medical/demographic differences between patients. What we need to think of next is how we should build our classifier, I'll assume that the current data is representative of the underlying population and appointment administration criteria (which is not known to us); this means that if 'no show' frequency for a given feature or a categorical level (example: 'MONTE BELO' of categorical feature neighborhood) is high then we expect it to hold true on new unseen data given the same conditions.

While we still can't extract a meaningful explanation or a common pattern that clearly discriminate a 'no show' appointment among all patients, we were able to note repeating patterns among some patients. We can think of these patterns as a hint to the underlying **real** cause of 'no show'; maybe these appointments are of a common medical specialty or relate to a specific venue/physician.

In the light of what we know so far, we can think of creating some additional features such as:

- Extracting date and time details for each appointment
- Calculate number of missed appointments per a given scheduling slot (date) collectively and on neighborhood level
- Assign a type to each appointment either follow-up / substitute
- Calculate difference between current and previous appointment scheduling hour
- Identify appointments scheduled on the same day
- Identify appointments held on the same day
- Target Encode neighborhood instead of discarding completely
- Combine neighborhood with other numeric features as a new categorical feature.
- Use Time stamps as a categorical feature! Think of it as a substitute for the missing details of each appointment (venue, type, medical specialization, physician, etc...), assuming that there is a relationship between time and these details.

Actually there are several ways to approach this particular problem, we can cluster the data and build separate model for each cluster. One drawback for this approach is that we might end up with several clusters and thus several independent models, or maybe treat it as a time series classification problem using dynamic time wrapping and K-nearest neighbor classifier but we do not have enough time data and segmentation of time data will be challenging; another approach is to make use of categorical features and combination of categorical/numeric features along with target encoding.

We will follow this last approach of categorical/numeric features and target encoding using RandomForest and CatBoost classifier, but first let’s extract these features.

<li><a href="#toc">Table of contents</a></li>

<a id='fengn'></a>

### Feature Engineering

At this point we should start splitting the data into train/test set and then preprocess each individually to avoid data leakage. In a nutshell, leakage occurs when:

- Test data is known (leaked) to the classifier during training; common example is scaling data using a scalar that is fitted on the entire dataset rather than being fitted on the training data set only. In that scenario the classifier will be trained on data that captures the distribution of values included in the testing set, so the testing set in that case lost its 'unseen' nature and became known to the classifier beforehand.

- Target variable is captured by the features used in training. Example is target encoding that is done wrong; this will be discussed in details later.

I'll not do the split now as I want to demonstrate the effect of leakage on prediction results; moreover, some of the preprocessing we will be doing does not pose leakage threat. However, I'll be highlighting preprocessing steps that can lead to leakage as we perform them.

#### Numeric Features

In [539]:
# Characteristics of scheduling day
df_hist['sch_day'] =  pd.to_datetime(df_hist.scheduling_day).dt.day
df_hist['sch_month'] =  pd.to_datetime(df_hist.scheduling_day).dt.month
df_hist['sch_hour'] =  pd.to_datetime(df_hist.scheduling_day).dt.hour
df_hist['day_of_week_sch'] =  pd.to_datetime(df_hist.scheduling_day).dt.dayofweek
df_hist['day_of_year_sch'] =  pd.to_datetime(df_hist.scheduling_day).dt.dayofyear

# Characteristics of appointment day
df_hist['app_day'] =  pd.to_datetime(df_hist.appointment_day).dt.day
df_hist['app_month'] =  pd.to_datetime(df_hist.appointment_day).dt.month
df_hist['day_of_week_app'] =  pd.to_datetime(df_hist.appointment_day).dt.dayofweek
df_hist['day_of_year_app'] =  pd.to_datetime(df_hist.appointment_day).dt.dayofyear

# Weekday or Weekend
# Week starts on Monday: Monday is o - Sunday is 6, Sat and Sun are weekends in Brazil
df_hist['sch_on_weekend'] =  df_hist.day_of_week_sch.apply(lambda x: 1 if x > 4 else 0)
df_hist['app_on_weekend'] =  df_hist.day_of_week_app.apply(lambda x: 1 if x > 4 else 0)

df_hist.head(1)

In [540]:
# aggregating features in order to profile the patient behavior

# cumulative previous appointments count
df_hist['previous_app'] = df_hist.groupby(['patient_id']).cumcount()

# cumulative missed appointments and absence frequency
df_hist['previous_missed_app'] = df_hist.groupby(['patient_id']).no_show.shift().fillna(0)
df_hist['previous_missed_app'] = df_hist.groupby(['patient_id']).previous_missed_app.cumsum()
df_hist['absence_frequency'] = (df_hist.previous_missed_app / df_hist.previous_app).round(4).fillna(0)

# same day appointment, multiple appointments on same day
df_hist['days_prev_app'] = df_hist.sort_values(['appointment_day', 'scheduling_day']).groupby(['patient_id']).appointment_day.diff().dt.days.fillna(0)
df_hist['days_fol_app'] = df_hist.sort_values(['patient_id', 'appointment_day']).groupby(['patient_id']).appointment_day.diff(-1).dt.days.fillna(0)
df_hist['same_day_app'] = df_hist.duplicated(['patient_id','appointment_day'], keep=False).astype('int')

# identify whether patient attended the previous appointment
df_hist['attended_previous_app'] = df_hist.groupby(['patient_id']).no_show.shift().replace({0:1, np.nan:0, 1:0}).astype('int')

# Time span between scheduling day
df_hist['norm_day'] = df_hist.scheduling_day.dt.normalize()

df_hist['schd_days_same'] = df_hist.sort_values(['patient_id', 'scheduling_day']).groupby(['patient_id']).norm_day.diff().dt.days\
.apply(lambda x: 1 if x ==0 else 0).fillna(0)

df_hist['schd_days_same_'] = df_hist.sort_values(['patient_id', 'scheduling_day']).groupby(['patient_id']).norm_day.diff().shift(-1).dt.days\
.apply(lambda x: 1 if x ==0 else 0).fillna(0)

df_hist['schd_days_same'] = np.where((df_hist.schd_days_same == 0) & (df_hist.schd_days_same_ == 1),1,df_hist.schd_days_same)

# time span between scheduling hours, measured in seconds
df_hist['same_day_count'] = df_hist[(df_hist.patient_id.duplicated(keep=False)) & (df_hist.schd_days_same ==1)]\
.sort_values(['patient_id', 'scheduling_day']).groupby(['patient_id', 'sch_day', 'sch_hour']).scheduling_day.transform('count')

data = df_hist[(df_hist.patient_id.duplicated(keep=False)) & (df_hist.schd_days_same ==1) & (df_hist.same_day_count >1)]\
.sort_values(['patient_id', 'scheduling_day'])

df_hist['schd_seconds_diff'] = 0

df_hist.loc[data.index, 'schd_seconds_diff'] = df_hist.sort_values(['patient_id', 'scheduling_day'])\
.groupby(['patient_id', 'sch_day', 'sch_hour']).scheduling_day.transform('diff').dt.seconds.bfill()

df_hist.schd_seconds_diff.fillna(0, inplace=True)

# Marking appointments scheduled within 10 min on the same day
df_hist['tight_schedule'] = np.where((df_hist.schd_seconds_diff > 0) & (df_hist.same_day_count > 1) & (df_hist.schd_seconds_diff <= 600), 1,0)

df_hist.head(1)

In [541]:
# Sample of appointments
df_hist[df_hist.patient_id == '111124532532143.0']

Leakage alert:
- 'absence_frequency' feature is calculated using 'no_show' column which is the target variable we are trying to predict, however, we are excluding current appointment from calculation as we do when target encoding a categorical feature using leave one out approach, this approach can still lead to overfitting.

As we can see in the sample of records above the feature captures the historical behavior of that patient excluding his last appointment.

In [542]:
# Tagging appointments either being a follow-up / substitute, any better idea than the below approach?

# First capture scheduling days that are same as appointment days (regardless of chronological sequence, 
# no separation yet between follow-up or substitute)
mark = df_hist.groupby('patient_id').apply(lambda x: x.scheduling_day.dt.date.isin(x.appointment_day.dt.date).astype('int'))

df_hist['mark'] = mark.reset_index().drop(columns=['patient_id', 'level_1'])

# filter out same day scheduling and appointments (appointments scheduled and attended on same date) as we want to focus
# on current scheduling and previous appointments
df_hist.mark = np.where((df_hist.mark == 1) & (df_hist.elapsed_days_sch == 0),0, df_hist.mark)

# capture scheduling days that are same as previous appointment days 
# we compare current scheduling to immediate previous one not all as in earlier mark
mark_2= df_hist.groupby('patient_id').apply(lambda x: x.scheduling_day.dt.date == x.appointment_day.shift().dt.date).astype('int')

df_hist['mark_2'] = mark_2.reset_index().drop(columns=['patient_id', 'level_1'])

# substitute appoints are the ones where current scheduling is same as immediate previous missed appoint day 
df_hist['substitute_app'] = np.where((df_hist.mark == 1) & (df_hist.mark_2 == 1) & (df_hist.attended_previous_app == 0),1, 0)

# follow-ups are rest of marked days
df_hist['follow_up_app'] = np.where((df_hist.mark == 1) & (df_hist.substitute_app == 0),1, 0)

# multiple follow-up appointments scheduled at same time
df_hist['mult_fol_app'] = df_hist[df_hist.follow_up_app == 1].groupby(['patient_id', 'norm_day']).follow_up_app.transform('count')\
.apply(lambda x: 1 if x >1 else 0)

df_hist.mult_fol_app.fillna(0, inplace=True)

df_hist.head(1)

In [543]:
# average elapsed days per neighborhood
df_hist['avg_elaps_day_neighbourhood'] = df_hist.groupby('neighbourhood').elapsed_days_sch.transform('mean').round()

# difference between elapsed day and average per neighborhood
df_hist['elday_navg_diff'] = df_hist.elapsed_days_sch - df_hist.avg_elaps_day_neighbourhood

df_hist.head(1)

In [544]:
# historical missed appointments per scheduling hour of the day and per scheduling hour per neighborhood
# to capture if a specific scheduling slot experience higher no_show frequency than another

df_hist['msd_app_sch_slot_nb'] = df_hist.sort_values(['appointment_day','scheduling_day']).groupby(['neighbourhood', 'sch_day', 'sch_month', 'sch_hour'])\
.no_show.shift().fillna(0)

df_hist['msd_app_sch_slot_nb'] = df_hist.sort_values(['appointment_day','scheduling_day']).groupby(['neighbourhood', 'sch_day', 'sch_month', 'sch_hour'])\
.msd_app_sch_slot_nb.cumsum().fillna(0)

# historical missed appointments per scheduling hour of the day
df_hist['msd_app_sch_slot'] = df_hist.sort_values(['appointment_day', 'scheduling_day']).groupby(['sch_day', 'sch_month', 'sch_hour'])\
.no_show.shift().fillna(0)

df_hist['msd_app_sch_slot'] = df_hist.sort_values(['appointment_day', 'scheduling_day']).groupby(['sch_day', 'sch_month', 'sch_hour'])\
.msd_app_sch_slot.cumsum().fillna(0)

# drop temp columns
df_hist.drop(columns = ['norm_day', 'schd_days_same_', 'mark', 'mark_2', 'same_day_count'], inplace=True)

# Sort columns
no_show_idx = df_hist.columns.get_loc("no_show")

columns_list = df_hist.columns.tolist()

columns_sort = columns_list[0:no_show_idx] + columns_list[no_show_idx+1:] + columns_list[no_show_idx : no_show_idx+1]

df_hist = df_hist[columns_sort]

df_hist.head(1)

Leakage alert:
- both features(msd_app_sch_slot' and 'msd_app_sch_slot_nb') are created using same method as 'absence_frequency' discussed earlier.

In [545]:
# Remove long elapsed days that belong to -ve class
df_hist = df_hist[~df_hist.index.isin(df_hist[(df_hist.elapsed_days_sch >90) & (df_hist.no_show ==0)].index)]

df_hist.reset_index(inplace=True, drop=True)

df_hist

<li><a href="#toc">Table of contents</a></li>

<a id='bline'></a>

#### Establishing baseline and analysing results

Now that we've extracted some features, it’s time to establish a baseline before going any further. Baseline is usually established before feature engineering but I want to have a measure of performance improvement before and after categorical feature extraction so we will establish it now.

We will be using RandomForest classifier and f1 score as a performance improvement metric while keeping an eye on recall, I'm seeking a balanced classification performance with high recall without significant loss in precession rather than a skewed one as in the previous attempt where the classifier generated large amount of false positive predictions.

In [546]:
# Model building and evaluation
class Results:
    def __init__(self, model, pred_p, pred_c, pr_precision, pr_recall, pr_thresh, pr_f1, pr_auc, fpr, tpr, roc_thresh, roc_auc):
        self.model = model
        self.pred_p = pred_p
        self.pred_c = pred_c
        self.pr_precision = pr_precision
        self.pr_recall = pr_recall
        self.pr_thresh = pr_thresh
        self.pr_f1 = pr_f1
        self.pr_auc = pr_auc
        self.fpr = fpr
        self.tpr = tpr
        self.roc_thresh = roc_thresh
        self.roc_auc = roc_auc
        
def train_eval(model, features_train, labels_train, features_test, labels_test):

    ''' 
    Train model and display several evaluation metrics including:
    
    1 - Area Under the Curve - Precision Recall Curve (PR)
    2 - Area Under the Curve - Receiver Operating Characteristics (ROC)
    3 - F measure
    4 - Thresholds for both curves
    5 - Precision
    6 - Recall
    7 - True Positive Rate
    8 - False Positive Rate 
    
    Both raw predictions and probabilities are also calculated
    '''

    t0 = time()
    model.fit(features_train, labels_train)
    print("training time:", round((time()-t0) / 60, 2), "m", '\n', '-' * 40)

    # Predict 

    t0 = time()
    pred_p = model.predict_proba(features_test)
    pred_c = model.predict(features_test)
    print("prediction time", round((time()-t0) / 60, 3), "m", '\n', '-' * 40)

    # Hold predictions of +ve class
    pred_p = pred_p[:,1]

    # Display results

    pr_precision, pr_recall, pr_thresh = precision_recall_curve(labels_test, pred_p)
    pr_f1, pr_auc = f1_score(labels_test, pred_c), auc(pr_recall, pr_precision)

    fpr, tpr, roc_thresh = roc_curve(labels_test, pred_p)
    roc_auc = auc(fpr, tpr)

    print('PR_AUC: ', pr_auc.round(4), '\n', '-' * 40, '\n', 'F1 score: ', pr_f1.round(4), '\n', '-' * 40, '\n', 'ROC_AUC: ', roc_auc.round(4), '\n', '-' * 40, '\n',
          classification_report(labels_test, pred_c, target_names=['show', 'no_show']), '-'*40, '\n', confusion_matrix(labels_test,  pred_c))

    return Results(model, pred_p, pred_c, pr_precision, pr_recall, pr_thresh, pr_f1, pr_auc, fpr, tpr, roc_thresh, roc_auc)

# Plotting feature importance
def rf_plot(model, features):
    
    ''' Plot feature importance of RandomForest Classifier'''
    
    fig  = plt.subplots(figsize=(10, 10))
    plt.tight_layout()

    feature_importance = model.feature_importances_
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + 0.5
    plt.barh(pos, feature_importance[sorted_idx], align="center")
    plt.yticks(pos, np.array(features.columns)[sorted_idx])
    plt.title("Feature Importance RF - Medical appointments")
    plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

In [547]:
# Establishing a baseline

# convert non numeric columns to string
df_hist.scheduling_day, df_hist.appointment_day = df_hist.scheduling_day.astype('str'), df_hist.appointment_day.astype('str')

# separate categorical and numeric columns
cat_cols = df_hist.select_dtypes('object').columns
num_cols = df_hist.columns[~df_hist.columns.isin(cat_cols)]

# unifying numeric dtypes
df_hist[num_cols[:-1]] = df_hist[num_cols[:-1]].astype(float, errors='ignore')

# split data
features = df_hist[num_cols].drop('no_show', axis=1)
labels = df_hist[num_cols].no_show

X_train, X_test, y_train, y_test = train_test_split(features, labels, stratify = labels, test_size=.2, random_state=42)

# Basline Model
rf_results = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), X_train, y_train, X_test, y_test)

The classifier did not perform well. We are particularly interested in f1-score and PR_AUC metrics as the dataset is imbalanced, both metrics displays low values as PR_AUC is below 50% and f1-score is almost 20%; One thing to observe that some metrics can be misleading when evaluating model performance on imbalanced datasets such as accuracy and ROC_AUC as both are showing high values 80% and 76% respectively but are shadowed by classification results of the -ve class with larger sample size.  

The classifier is having poor recall on the +ve class (no_show) as only 355 out of 2928 no_show samples were correctly classified, one reason is that +ve class is underrepresented another is that current features does not provide decent class discrimination as we've discussed earlier. 

Note that we've partially addressed class imbalance issue on classifier level by setting class_weight hyperparameter to 'balanced', this will force the classifier pay more attention to +ve class.

Now that we have a baseline to measure against, let's look at feature importance.

In [548]:
# plotting feature importance
rf_plot(rf_results.model, X_test)

Some of the features we've just created are at the top of feature importance. key observation from the above plot is that features that capture time of appointments and related variations are among the top contributors to prediction results.

Before moving on to Categorical feature engineering lets visualize class distribution using all current features

In [549]:
def dr_plot(df, df_idx, labels, pca = True, undersample = False, undersample_method = None, plot_only = True):

    '''
    Reduce dimensionality to visualize data using either PCA or T-SNE, will construct a dataframe of reduced dimensions 
    indexed same as main dataframe for ease of access. Metric used for T-SNE is cosine.

    df: features scaled
    df_idx: main dataframe to capture index before scaling
    labels: target/class
    method: PCA or T-SNE, default PCA
    undersample_method: fitted undersampler, used to capture index of kept datapoints
    plot_only: True is for visualization only, else returns both fitted pca and constructed dataframe if pca; and dataframe only for tsne

    '''

    if pca:

        pca = PCA(n_components=2, random_state = 45) # create a PCA object
        pca.fit(df) # do the math
        pca_data = pca.transform(df) # get PCA coordinates for scaled_data

        fig, axes = plt.subplots(figsize=(30, 15))   

        if undersample:
            pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2'], index = df_idx.iloc[undersample_method.sample_indices_].index)
            us_method_name = str(undersample_method).split('(')[0]
            print(f'Explained Variance Ratio for PC1 and PC2 respectively - {us_method_name}', np.round(pca.explained_variance_ratio_, 2)) 

            _ = plt.scatter(pca_df.PC1, pca_df.PC2, c =  labels, marker='x', cmap = 'RdYlBu')

            plt.title(f'Medical Appointments PCA - {us_method_name}')

        else:
            pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2'], index = df_idx.index)
            print('Explained Variance Ratio for PC1 and PC2 respectively', np.round(pca.explained_variance_ratio_, 2))

            _ = plt.scatter(pca_df.PC1, pca_df.PC2, c =  labels, marker='x', cmap = 'RdYlBu')

            plt.title(f'Medical Appointments PCA')

        plt.legend(handles = _.legend_elements()[0], labels = ['Show', 'No_show'])
        plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

        if not plot_only:
            return [pca, pca_df]

    else:
        tsne = TSNE(n_components = 2, init='pca', learning_rate='auto', metric='cosine', square_distances=True, random_state = 45)
        tsne_data = tsne.fit_transform(df)

        fig, axes = plt.subplots(figsize=(30, 15))   

        if undersample:
            tsne_df = pd.DataFrame(tsne_data, columns=['x', 'y'], index = df_idx.iloc[undersample_method.sample_indices_].index)
            us_method_name = str(undersample_method).split('(')[0]

            _ = plt.scatter(tsne_df.x, tsne.y, c =  labels, marker='x', cmap = 'RdYlBu')

            plt.title(f'Medical Appointments T-SNE - {us_method_name}')

        else:
            tsne_df = pd.DataFrame(tsne_data, columns=['x', 'y'], index = df_idx.index)

            _ = plt.scatter(tsne_df.x, tsne_df.y, c =  labels, marker='x', cmap = 'RdYlBu')

            plt.title(f'Medical Appointments T-SNE')

        plt.legend(handles = _.legend_elements()[0], labels = ['Show', 'No_show'])
        plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

        if not plot_only:
            return tsne_df

In [550]:
# Visualization using PCA for dimensionality reduction

data = MaxAbsScaler().fit_transform(df_hist[num_cols[:-1]])

mask_pca = dr_plot(data, df_hist, df_hist.no_show, plot_only = False)

We are looking at all appointments using two Principle Components (PC), PCA is a dimensionality reduction and feature engineering method where we can reduce all features in the dataset down to a selected number of principle components for visualization or even use these new components as a training data instead of original features.

Important thing to note is that PCA is an unsupervised linear dimensionality reduction technique; the principal components produced are mixture of original features. Each component does explain some of the variation in the data and usually the first two components are the top contributors but this depends on the data and nature of linear relationship among features/variables.

So what we see on the plot is how close/far each appointment is from one another based on these two principle components (collection of features) which do not provide significant explanation for variation within the data (only 26% explained variance ration). We are using MaxAbs scaler to preserve the sparsity of the data and it is apparent that there are some regions (appointments) having a clear separation of +ve and -ve class while others are not (class overlap); check cords x: 0.6 and y: 1.2.

PCA may not be an idle 2D visualization technique for our data given the low explained variance ratio of the first two components but it is enough to serve the purpose of having a quick overview of the dataset.

In [551]:
# visualize the loading scores

plt.matshow(mask_pca[0].components_, cmap='RdYlBu')
plt.yticks([0, 1], ["PC1", "PC2"])
plt.colorbar()
plt.grid(None)
plt.xticks(range(len(df_hist[num_cols[:-1]].columns)), df_hist[num_cols[:-1]].columns, rotation=90, ha='center')
plt.xlabel("Feature")
plt.ylabel("Principal components");

One way to understand PCA plot is to visualize the contribution of each feature to the components being analysed, PC1 and PC2. Contribution is denoted by how correlated each feature to the component, for example appointments having higher values of PC2 axis (y-axis) represents appointments of male patients that did not receive sms or attend/have a previous appointments while having high absence score and being scheduled in the same day with narrow scheduling window.

While these features combined could provide good class discrimination among some of the appointments, this does not hold true for the majority of appointments as we noted earlier inspecting the historical records of some patients; what works for some patients doesn't work for others.

In [552]:
# Zoom in a dense no_show area
pca_df = mask_pca[1]
subset = pca_df[(pca_df.PC1 > .6) & (pca_df.PC2 > 1.2)]
labels_ = df_hist[df_hist.index.isin(subset.index)].no_show

# plotly for indexing and inspecting specific points
fig = subplots.make_subplots(rows=1, cols=1)

t = go.Scatter(x=subset.PC1, y=subset.PC2, mode='markers', showlegend=False, marker=dict(size=10, color=labels_, 
                 colorscale='RdYlBu', line=dict(color='black', width=1)),
                 hovertemplate = '<br>X</b>: %{x}' + '<br>Y</b>: %{y}<br>'+ '<br>Index: %{text}', text=subset.index)
                  
fig.append_trace(t, 1, 1)
fig.update_layout(autosize=False, width=1200, height=700)

We are now looking at a subset from the PCA plot having dense 'no show' appointments, we can observe three clusters of appointments each having its own characteristics depending on where it lies on x and y axis, We will select two of the adjacent appointments (x: 0.85, y: 1.44) and closely inspect full appointment details trying to figure out ways to better separate them but first let’s look and the characteristics of this subset.

In [553]:
# full appointment details of a selected subset
detailed_subset = df_hist[df_hist.index.isin(subset.index)]

detailed_subset.describe()

The descriptive summary corroborates with our earlier interpretation when we visualized feature contribution to each principal component. 'No show' mean for these appointments is 60% and they are mainly related to male patients (76%) that did not receive sms or attend/have a previous appointments while having high absence frequency (68%), additionally most appointments are being scheduled on the same day with narrow scheduling window (couple of seconds between each scheduling time) and are scheduled to take place on days having multiple appointments (several appointments on same day).

In [554]:
detailed_subset.head()

In [555]:
# full records of patients having appointments in the selected subset
df_hist[df_hist.patient_id.str.contains('11474894783.0|118375575136787.0')]

100% 'no show', almost all appointments for these two patients are being scheduled on the same day with narrow scheduling window (split seconds between each scheduling time) and are multiple appointments on same appointment day.

In [556]:
# full appointment details of selected adjacent points having different class labels (x: 0.85, y: 1.44)
detailed_subset.loc[[57346, 29863]]

We can note that most of numeric features are similar, it is the time, neighborhood and age related features that make clear distinction between the two.

<li><a href="#toc">Table of contents</a></li>

<a id='fengc'></a>

#### Categorical Features

Previously I've discarded neighborhood completely, now will try to make use of it and check classification performance. From now on we will be creating new features then evaluate model performance right after and compare results to the baseline model.

In [557]:
# check cardinality of neighborhood feature

print(f'Cardinality of Neighbourhood feature is: {df_hist.neighbourhood.nunique()}')

In [558]:
# least represented neighborhood
df_hist.groupby('neighbourhood').no_show.mean().sort_values().head(1), df_hist.groupby('neighbourhood').no_show.count().sort_values().head(1)

Cardinality refers to the number of unique records (levels) in a given categorical feature; neighborhood feature does have 79 unique levels.

There are several ways to handle categorical (cat) features; all have the same basic principle that is seeking a unique numeric representation for each category. Below are some of the popular ways for handling such features:

- A) Dropping: same as I did in first attempt. This approach is good if we are sure that cat features do not bring any value to the classification results, like the gender feature in this dataset.
- B) One-hot Encoding (ohe): will produce a sparse matrix for each unique category. For neighborhood feature we would generate 79 additional columns each representing a single level (neighborhood).
- C) Label Encoding: substituting each level with a unique value (number). For neighborhood feature we would represent each neighborhood by a unique number ranging from 0-78, no additional columns are going to be generated as the numbers substitutes the string values in the same columns.
- D) Target Encoding: very powerful yet very dangerous encoding technique as it can lead to serious data leakage and misleading optimistic performance. Each category (level) is substituted by the frequency of occurrence of the +ve class in that level, the frequency can take any form such as mean, count, etc..
    - Example: we calculate frequency of 'no show' appointments per each level neighborhood feature. I'll be using the mean as an indication of frequency which is simply the +ve class samples / all samples per level.
 
Why it can lead to misleading results?

- In the example above, neighborhood 'AEROPORTO' have a mean frequency of 0 calculated based on 2 samples only. If we encode this neighborhood as it is then we would be replacing it with the target variable we are trying to predict as the 2 appointments were both attended (0 in target value); another important thing is that we are not confident enough in the calculated statistic(mean) as it is based on 2 samples only.

How to deal with these issues?

- One approach is to introduce noise, this can take many forms but I'll be using the global mean of 'no show' as a substitute for the calculated mean per each level.

In [559]:
# likelihood of absence per neighborhood
df_hist.groupby('neighbourhood').no_show.mean().sort_values().head(50)

In [560]:
# checking distribution of no_show across neighborhoods
df_hist.groupby('neighbourhood').no_show.mean().plot(kind='hist', bins =20);

In [561]:
# sample of appointments per selected neighborhoods
df_hist[(df_hist.neighbourhood == 'REDENÇÃO')].no_show.count(), df_hist[(df_hist.neighbourhood == 'JARDIM CAMBURI')].no_show.count()

One way to target encode cat feature is to compute the probability of 'no show' at each level (per each neighborhood) and use it as a 'score' indicating 'no show' per neighborhood.

Few problems need to be addressed if we are to follow this approach:

A) With few 'no show' samples and imbalanced nature of the dataset, some neighborhoods are going to have 'no show' probability of zero (example: AEROPORTO) making the distribution of the new feature skewed towards zero. Luckily we do not have many examples of such problem in this feature.

B) Calculated probability could be the same for neighborhoods regardless samples available. For example, 'no show' probability for the two neighborhoods 'REDENÇÃO' and 'JARDIM CAMBURI' is approximately 19%, however, it is calculated based on a sample of 1k and 5k appointments respectively for each neighborhood. Therefore, we are more confident in the 'no show' likelihood statistic for 'JARDIM CAMBURI' than the other neighborhood.

To solve these issues, we're going to combine a priori of 'no show' with the calculated 'no show' probability per neighborhood. The idea is that when there’s not much information (neighborhoods with few samples/appointments) we want to encode these neighborhoods as closely as possible to the mean probability of 'no show' across all neighborhoods (global mean). On the other hand, when we have enough information (more samples/appointments) we want to consider the 'no show' probability per that neighborhood as it is. In order to achieve this, we are going to assign a weight to each calculated probability so that it scales with the available samples from which the statistic is calculated, this weight will pull encoded statistic towards global one in case of few samples and vise versa.

In [562]:
# Target encoding cat variables

def target_enc_scoring(df, feat, label, log_trans = False, with_weight = True):
    ''' Calculate scores for each level of a categorical feature and adjust the score by sample weight'''
    
    # incase single column is passed to encode instead of several ones
    if isinstance(feat, str):
        feat = [feat]
    else:
        feat = feat
    
    # Compute the global mean
    priori = df[label].mean()

    # total samples
    tot_samples = df.shape[0]
    
    for f in feat:

        # Compute frequency, mean and weight of each level
        agg = df.groupby(f)[label].agg(['count', 'mean'])
        
        # no show likelihood
        means = agg['mean']
        
        # sample count per each level
        if log_trans:
            counts = np.log(agg['count'])
        else:
            counts = agg['count']
        
        weight = counts / tot_samples
            
        # scale weights to reduce influence of the priori
        # if we do not scale, the priori will always have higher influence on the calculated statistic
        weight = MinMaxScaler().fit_transform(weight.values.reshape(-1,1))

        # Compute the scoring function
        scoring = (weight.ravel() * means) + ((1-weight.ravel()) * priori)

        # add calculated score to dataframe
        if with_weight:    
            df[f'{f}_score'] = df[f].map(scoring)
        
        # frequency without weight - if you are feeling adventurous
        else:
            df[f'{f}_score'] = df.groupby(f)[label].transform('mean')

In [563]:
# Target encode
target_enc_scoring(df_hist, 'neighbourhood', 'no_show')

df_hist.head(1)

In [564]:
# global no_show mean
round(df_hist.no_show.mean(), 2)

In [565]:
df_hist[(df_hist.neighbourhood == 'REDENÇÃO') |  (df_hist.neighbourhood == 'JARDIM CAMBURI')].neighbourhood_score.value_counts()

Initially both neighborhoods had same 'no show' probability, after adjusting weights the probability of 'JARDIM CAMBURI' stayed the same while that of 'REDENÇÃO' moved closer to the priori (global one). It makes sense as we would be more confident in the calculated 'no show' probability for 'Jardim' than that of 'Red' given the sample size.

In [566]:
# Better results already?

# updated train and test sets with new features
X_train = pd.concat([X_train, df_hist[df_hist.index.isin(X_train.index)][['neighbourhood_score']]], axis = 1, 
                    join= 'inner')
X_test = pd.concat([X_test, df_hist[df_hist.index.isin(X_test.index)][['neighbourhood_score']]], axis = 1, 
                    join= 'inner')

rf_results = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), X_train, y_train, X_test, y_test)

Better precision but no change in recall, not what we are looking for.

In [567]:
# Extracting additional time features
df_hist['time_stamp_sch'] = pd.to_datetime(df_hist.scheduling_day).dt.time.astype('str')

# Remove seconds from time stamp
df_hist['sch_time'] = df_hist.time_stamp_sch.apply(lambda x: str(x)[:-3]).astype('str')

# # Update cat cols
# cat_cols = df_hist.select_dtypes('object').columns
# df_hist[cat_cols] = df_hist[cat_cols].astype('str')

# Target encode
target_enc_scoring(df_hist, ['time_stamp_sch', 'sch_time'], 'no_show')

df_hist.head(2)

In [568]:
# updated train and test sets with new features
X_train = pd.concat([X_train, df_hist[df_hist.index.isin(X_train.index)][['time_stamp_sch_score', 'sch_time_score']]], axis = 1, 
                    join= 'inner')
X_test = pd.concat([X_test, df_hist[df_hist.index.isin(X_test.index)][['time_stamp_sch_score', 'sch_time_score']]], axis = 1, 
                    join= 'inner')

rf_results = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), X_train, y_train, X_test, y_test)

Big leap in recall while maintaining balanced performance overall.

In [569]:
# combine neighborhood with time features
df_hist["nbr_sch_stamp"] = (df_hist.neighbourhood + "_" + df_hist.time_stamp_sch.astype('str'))

df_hist["nbr_sch_time"] = (df_hist.neighbourhood + "_" + df_hist.sch_time.astype('str'))

# Target encode
target_enc_scoring(df_hist, ['nbr_sch_stamp','nbr_sch_time'], 'no_show')

df_hist.head(2)

In [570]:
# updated train and test sets with new features
X_train = pd.concat([X_train, df_hist[df_hist.index.isin(X_train.index)][['nbr_sch_stamp_score', 'nbr_sch_time_score']]], axis = 1, 
                    join= 'inner')
X_test = pd.concat([X_test, df_hist[df_hist.index.isin(X_test.index)][['nbr_sch_stamp_score', 'nbr_sch_time_score']]], axis = 1, 
                    join= 'inner')

rf_results = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), X_train, y_train, X_test, y_test)

Another big leap in recall combined with balanced performance overall, but hold your horses because we have leakage!

So far we've been using a fixed train/test sets that is split before target encoding, then we update the train and test set with the new encoded features that are calculated from the main dataframe and thus the calculated statistic (no_show mean per level of categorical feature) takes into account all samples (train and test samples before split) which means that the train set 'knows' about target values of test set in an indirect way, that’s why any preprocessing that includes a statistic calculation (e.g.: standardization/normalization, target encoding, etc..) needs to be performed on training data first after splitting then transformed to the test set; otherwise data leaks. 

To summarize, treat test set as a completely independent set that is transformed based on calculations made on the training set.

What is strange though is that we did not achieve perfect scores on test set even with leakage! Is this actually a leakage issue or the new features are just so powerful at discriminating ‘no show’ appointments? Could it be the effect of the regularization we applied when target encoding using sample weight?

Let’s first look at feature importance, maybe we find out the reasons for the big leaps in classification results or maybe it reveals towards the end of the notebook.

<a id='pm'></a>

In [571]:
# Feature Importance
rf_plot(rf_results.model, X_test)

Usually this is when we should start to panic! seeing one feature that is target encoded dominates the feature importance meter is alarming, however, some of the other features that are encoded in the same manner are not that influential additionally even though we have leaked data into training set classification performance did not improve dramatically, well to some extent!

<li><a href="#toc">Table of contents</a></li>

In [572]:
# Investigate time stamp feature
df_hist.groupby('time_stamp_sch').no_show.value_counts().sort_values(ascending=False).head(10)

In [573]:
# zoom in a specific date
print(round(df_hist[df_hist.time_stamp_sch == '17:17:46'].no_show.mean(),2))
df_hist[df_hist.time_stamp_sch == '17:17:46'].sort_values(['patient_id', 'neighbourhood'])

The discriminatory power of this feature makes some sense now, it appears that specific scheduling hours experience significantly higher no_show rate than others; but why?

Well, given the consistent occurrence of no_show (91%) at this specific timestamp across a range of patients from different neighborhoods and age groups it must be reflecting an underlying criteria contributing to the no_show behavior, maybe all relate to specific medical provider/nature of visit/medical specialization/physician or maybe the fact that all appointments are on Monday (start of the week) combined with another reason mentioned earlier.

So, prediction results are reflecting realistic performance? No, because we still have leakage, but we now know that these new features are good at separating class labels and that our target encoding method is properly regularized, is this true?

In [574]:
# check record of a selected patient from the above time frame
df_hist[df_hist.patient_id == '12143623887377.0'].sort_values(['appointment_day', 'scheduling_day'])

By examining the detailed records of one of the patients from the above scheduling window, we note that appointments are awkwardly scheduled (scheduled on the same day with split seconds difference), and the patient does appear to be missing all such appointments. This is actually true for all 'no show' appointments in that specific window, check the 'tight schedule' feature. We observed this behavior several times earlier among different patients who miss some/all appointments that are scheduled in a very narrow time span, we can't actually tell if these are erroneous records or they are actual appointments and why such practice is taking place.  

If I was working on this project in a real life setup I'd stop here and investigate these specific dates further and incorporate more data about each appointment which should be readily available with whoever is responsible for administering the process of scheduling appointments; but we will continue just for the sake of satisfying my curious soul and also to explore the possibility of building a predictive model out of this dataset.

Now that we have examined our suspicious feature, back to the leakage problem; before handling this problem we need to address another problem that popped up during our investigation of time stamps above, which is absence of uniform distribution categorical levels (time stamps have few frequent levels and many rare levels). We know that this will be problematic if we are to continue target encoding categorical features as we discussed earlier even with the introduction of noise; so we are going to group all these unique samples into a single level.

Let's first examine how our target encoding function works.

In [575]:
# how does the encoding function works?

priori = df_hist.no_show.mean()
tot_samples = df_hist.shape[0]

# Compute frequency, mean and weight of each level
agg = df_hist.groupby('time_stamp_sch')['no_show'].agg(['count', 'mean'])
# sample count per each level
counts = agg['count']
# no show likelihood
means = agg['mean']
# proportion of transactions per level to all the transactions
# how confident should we be in calculated statistic?
weight = counts / tot_samples

# scale weights to reduce influence of the priori
# if we do not scale, the priori will always have higher influence on the calculated statistic
weight = MinMaxScaler().fit_transform(weight.values.reshape(-1,1))

# Compute the scoring function
scoring = (weight.ravel() * means) + ((1-weight.ravel()) * priori)

In [576]:
# check results
temp_df = scoring.to_frame().rename(columns={'mean':'scoring'})
temp_df = pd.concat([agg,temp_df], axis =1).reset_index()
temp_df['weight'] = weight

temp_df.head()

In [577]:
# frequency of levels
temp_df['count'].value_counts()

few frequent levels but many rare levels

In [578]:
# sample of frequent levels
temp_df[temp_df['count'] > 20]

In [579]:
# sample of rare levels
temp_df[temp_df['count'] == 1].head()

In [580]:
# weights distribution
temp_df.weight.plot(kind='hist', bins =20);

In [581]:
# scores distribution
temp_df.scoring.plot(kind='hist', bins =20);

We can see how the scoring (target encoding) function works, when we have many samples the scores(mean encoding) are pulled towards the true target probability(mean) per level unlike when samples are rare that’s when scores tend to be closer to the global target probability.

This address the problems associated with lack of samples to some extent and we can see that encoded scores are normally distributed despite the skew in weights. However, a new problem emerges being absence of uniform distribution of categorical levels; there are too many rare levels compared to frequent ones this means that:

- Majority of scores do not capture the trend we intend it to capture (fluctuations in 'no show' among levels) because simply it is dominated by the global mean representing these rare levels and we can see how 'narrow' is the distribution of encoded scores.

- Training set will not be representative and the model will overfit, simply because there will inevitably be some levels that appear in testing set and are not seen by the model during training, also the model will overfit the training data being trained on a single sample for each level.

So we are going to group these rare levels into a single level.

In [582]:
# Combining best features with target
df_hist['time_stamp_sch_target'] = (df_hist.time_stamp_sch.astype(str) + '_' + df_hist.no_show.astype(str))
df_hist['nbr_sch_time_target'] = (df_hist.nbr_sch_time.astype(str) + '_' + df_hist.no_show.astype(str))

# number of samples per each combined category
df_hist['time_stamp_sch_target_sample_count'] = df_hist.groupby('time_stamp_sch_target').no_show.transform('count')
df_hist['nbr_sch_time_target_sample_count'] = df_hist.groupby('nbr_sch_time_target').no_show.transform('count')

df_hist.head()

In [583]:
# statistics of cat features
feat = ['time_stamp_sch', 'nbr_sch_time']

for f in feat:
    sample_size = sorted(df_hist[f'{f}_target_sample_count'].unique())
    for s in sample_size:
        # statistics per sample size
        print(f'feature \'{f}_target\': ', f'\'no_show\' mean of levels having {s} sample(s) :', round(df_hist[df_hist[f'{f}_target_sample_count'] == s].no_show.mean(),2),\
              '---- total sample size per level :', df_hist[df_hist[f'{f}_target_sample_count'] == s].no_show.count())

What’s interesting to note is that levels having single samples are abundant and also have high 'no show' rate compared to other levels. For the feature 'time_stamp_sch' the +ve examples of levels having single samples are around 24k while that of the scheduling window we inspected earlier are only 21 samples, this will affect how these levels are going to be represented in terms of average 'no show' as now, after grouping rare levels into a single group, the encoding function will treat the new group as the dominate one and others as being 'rare' having 'no show' mean closer to the global average.

Should we care? I'd refrain from using such approach as it solves one problem but creates another; however, the fact that these new levels will have high 'no show' rate than other, and thus be more distinguishable, is interesting to explore further so we are going to group levels having single samples only and leave the rest as is.

In [584]:
# grouping levels having single samples

for l in feat:                                    
    # statistics of unique samples (single samples)
    print(f'\'{l}_target\' unique samples \'no_show\' mean:', round(df_hist[df_hist[f'{l}_target_sample_count'] == 1].no_show.mean(),2),\
          '---- sample count:', df_hist[df_hist[f'{l}_target_sample_count'] == 1].no_show.count())
    
    # grouping unique samples under one category
    df_hist.loc[df_hist[f'{l}_target_sample_count'] == 1, f'{l}_target'] = 'rare__'
    
    # update count
    df_hist[f'{l}_target_sample_count'] = df_hist.groupby(df_hist[f'{l}_target'])['no_show'].transform('count')
    
    # Remove association with target so the whole level is uniquely encoded regardless of target association
    df_hist[f'{l}_target'] = df_hist[f'{l}_target'].apply(lambda x: x[:-2])
    
# target encode
target_enc_scoring(df_hist, ['time_stamp_sch_target','nbr_sch_time_target'], 'no_show')

df_hist.head()

In [585]:
# closer look at what just happened

df_hist[['time_stamp_sch', 'nbr_sch_time', 'time_stamp_sch_score', 'nbr_sch_time_score', 
         'time_stamp_sch_target', 'nbr_sch_time_target', 'time_stamp_sch_target_score', 'nbr_sch_time_target_score']].head(10)

In [586]:
# number of samples below the selected rare sample threshold
print(temp_df[temp_df['count'] == 1].shape[0], df_hist.time_stamp_sch.nunique())

# earlier scores and weights
t = '08:45:03|11:13:45'
temp_df[temp_df.time_stamp_sch.str.contains(t)]

Since we are going to use target encoding for cat features we need to ensure that encoded scores are representative of underlying pattern and at the same time avoid the problems we discussed earlier, so we did the following for each level of a categorical feature:

- If there are enough samples (>1) per level, then pass.
- Else group all these unique samples under a unified level.

Changing the threshold for considering a level to be rare is going to affect final scores calculated and in turn prediction results, we've select a threshold of 1 samples per level given the large number of samples below that threshold (13K out of 32k for 'time_stamp_sch'). However, this introduced another problem for levels having samples above that threshold for example the levels '08:45:03' and '11:13:45' both have 0 'no show' appointments and their earlier encoding reflected such fact by having an average target encoded score below the global average but after grouping unique samples their scores are now same as the global mean hiding the fact that these two levels experienced a lower 'no show' rate. To overcome this we will apply log transformation to counts before calculating weights to control the effect of sample size of the new level on weights calculation and encoded score of each level.

Let's examine scores after log transformation.

In [587]:
# Final look at weights and encoded scores after grouping and log transformation
df = df_hist

priori = df.no_show.mean()
tot_samples = df.shape[0]

# Compute frequency, mean and weight of each level
agg = df.groupby('time_stamp_sch_target')['no_show'].agg(['count', 'mean'])
# sample count per each level
counts = np.log(agg['count'])

# no show likelihood
means = agg['mean']
# proportion of transactions per level to all the transactions
# how confident should we be in calculated statistic?
weight = counts / tot_samples

# scale weights to reduce influence of the priori
# if we do not scale, the priori will always have higher influence on the calculated statistic
weight = MinMaxScaler().fit_transform(weight.values.reshape(-1,1))

# Compute the scoring function
scoring = (weight.ravel() * means) + ((1-weight.ravel()) * priori)

# check results
temp_df_n = scoring.to_frame().rename(columns={'mean':'scoring'})
temp_df_n = pd.concat([agg,temp_df_n], axis =1).reset_index()
temp_df_n['weight'] = weight

In [588]:
# current scores and weights
temp_df_n[temp_df_n.time_stamp_sch_target.str.contains(t)]

If we compare to previous scores and weights before and after log transformation, we now have:
- Reduced weights
- Scoring closer to the 'no show' statistic per level and at the same time reflective of underlying sample size

In [589]:
# scores distribution
fig, axes = plt.subplots(ncols=2, figsize=(10, 5), constrained_layout=True)
ax1, ax2 = axes

temp_df.scoring.plot(kind='hist', bins =20, ax = ax1, title = 'Before grouping')
temp_df_n.scoring.plot(kind='hist', bins =20, ax = ax2, title = 'After grouping');

Scores are still centered on global mean but more spread.

<li><a href="#toc">Table of contents</a></li>

<a id='modeling'></a>

## Modeling

We now do a fresh start considering the notes from analysing previous classification results and related issues.

In [591]:
# Starting Fresh - Group rare levels, train/test split then apply target encoding

df_split = df_hist[df_hist.columns[:-15]].copy()

# Extracting additional time features
df_split['time_stamp_sch'] = pd.to_datetime(df_split.scheduling_day).dt.time.astype('str')

# Remove seconds from time stamp
df_split['sch_time'] = df_split.time_stamp_sch.apply(lambda x: str(x)[:-3]).astype('str')

# combine neighbourhood with time features
df_split["nbr_sch_stamp"] = (df_split.neighbourhood + "_" + df_split.time_stamp_sch.astype('str'))
df_split["nbr_sch_time"] = (df_split.neighbourhood + "_" + df_split.sch_time.astype('str'))

# number of samples per rare levels
df_split['time_stamp_sch_sample_count'] = df_split.groupby('time_stamp_sch').no_show.transform('count')
df_split['sch_time_sample_count'] = df_split.groupby('sch_time').no_show.transform('count')
df_split['nbr_sch_stamp_sample_count'] = df_split.groupby('nbr_sch_stamp').no_show.transform('count')
df_split['nbr_sch_time_sample_count'] = df_split.groupby('nbr_sch_time').no_show.transform('count')

# statistics of cat features
feat = ['time_stamp_sch', 'sch_time', 'nbr_sch_stamp', 'nbr_sch_time']

for f in feat:
    # Combining best features with target
    df_split[f'{f}_target'] = (df_split[f'{f}'].astype(str) + '_' + df_split.no_show.astype(str))
    
    # number of samples per each combined category
    df_split[f'{f}_target_sample_count'] = df_split.groupby(f'{f}_target').no_show.transform('count')
    
    # statistics of unique samples (single samples)
    print(f'\'{f}\' unique samples \'no_show\' mean:', round(df_split[df_split[f'{f}_target_sample_count'] == 1].no_show.mean(),2),\
          '---- sample count:', df_split[df_split[f'{f}_target_sample_count'] == 1].no_show.count())
    
    # grouping unique samples under one category
    df_split.loc[df_split[f'{f}_target_sample_count'] == 1, f'{f}'] = 'rare'
    
    # update count
    df_split[f'{f}_target_sample_count'] = df_split.groupby(df_split[f'{f}'])['no_show'].transform('count')
    
    # drop temp column
    df_split.drop(f'{f}_target', axis = 1, inplace = True)

# Sort column
no_show_idx = df_split.columns.get_loc("no_show")

columns_list = df_split.columns.tolist()

columns_sort = columns_list[0:no_show_idx] + columns_list[no_show_idx+1:] + columns_list[no_show_idx : no_show_idx+1]

df_split = df_split[columns_sort]

df_split.head()

In [592]:
# Train/Test split

# separate categorical and numeric columns
cat_cols = df_split.select_dtypes('object').columns
num_cols = df_split.columns[~df_split.columns.isin(cat_cols)]

# unifying numeric dtypes
df_split[num_cols[:-1]] = df_split[num_cols[:-1]].astype(float, errors='ignore')

# split data
features = df_split.drop(df_split.filter(regex='count|no_show').columns, axis=1)
# features = df_split.drop(['no_show'], axis=1)
labels = df_split.no_show

X_train, X_test, y_train, y_test = train_test_split(features, labels, stratify = labels, test_size=.2, random_state=42)

X_train.head()

In [593]:
# Target encode and transform test set

def target_encode_feat_mapping(key, X_train, X_test, y_train, enc_func = target_enc_scoring, with_weight = True, 
                               log_trans = False, fillna = True):
    
    '''apply target encoding to training set then transform test set'''
    
    # incase several columns are passed at one
    if isinstance(key, str):
        key = [key]
    else:
        key = key

    X_train['target'] = y_train
    
    # Last column in train set, to mark start of newly created columns (0 indexing)
    # -1 for no_show column to be dropped latter
    start_col = X_train.shape[1] - 1

    # Target encode
    enc_func(X_train, key, 'target', with_weight = with_weight, log_trans = log_trans)

    # drop
    X_train.drop(['target'], axis=1, inplace=True)
    
    # newly created target encoded columns of train set
    value = X_train.iloc[:,start_col:].columns
    
    # X_test transformation   
    # applying transformation and filling nan (unseen data) with average of training data columns
    for k, v in zip(key, value):
        # map keys(label encoded cat feats) to their target encoded values in the train set
        mapr = dict(X_train[[k, v]].values)
        
        if fillna:
            # filling nan with calculated scores mean
            X_test[v] = X_test[k].map(mapr).fillna(X_train[v].mean())
        else:
            X_test[v] = X_test[k].map(mapr)

            
# Copies to modify
X_train_mod = X_train.copy()
X_test_mod = X_test.copy()

key = ['time_stamp_sch', 'sch_time', 'neighbourhood', 'nbr_sch_stamp','nbr_sch_time']

target_encode_feat_mapping(key, X_train_mod, X_test_mod, y_train, log_trans = True)

# Remove cat features
X_train_mod = X_train_mod.drop(cat_cols, axis=1).copy()
X_test_mod = X_test_mod.drop(cat_cols, axis=1).copy()

X_test_mod.head()

In [594]:
# leakage busted?
rf_results = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), 
                        X_train_mod, y_train, X_test_mod, y_test)

Precision and recall are now 64 and 62 respectively, down from 76 and 64 before we deal with leakage issue. The former scores also incorporate the effect of grouping unique samples per each cat feature.

In [595]:
# Feature Importance
rf_plot(rf_results.model, X_test_mod)

Now we examine the effect of incorporating more categorical features to the model combining neighborhood and time features.

In [596]:
# Assessing combining neighborhood and elapsed_days_sch feature
df_split.groupby(['neighbourhood', 'elapsed_days_sch']).no_show.agg(['count', 'mean']).sort_values('mean', ascending=False).head(10)

In [597]:
# detailed records of combined features
df_split[(df_split.neighbourhood == 'HORTO') & (df_split.elapsed_days_sch == 10)]

Combining both features seems beneficial as some categories will have pure 'no show' appointments. In the example above, all appointments in neighborhood 'HORTO' having 10 elapsed days between scheduling and appointment dates were 'no show' and these are appointments of different patients so this behavior is not confined to a single patient. 

Exact reason of 'no show' is still vague as most of appointment details are somewhat similar; however, there is consistency among scheduling and appointment dates as all appointments were scheduled on the same day to take place on the same date so maybe they relate to specific medical provider or share some common criteria.

In [598]:
# combine neighborhood with elapsed days features
df_split['nbr_elapsed_days'] = (df_split.neighbourhood + "_" + df_split.elapsed_days_sch.astype('str'))

# combine with target for grouping
df_split['nbr_elapsed_days_target'] = (df_split.nbr_elapsed_days.astype(str) + '_' + df_split.no_show.astype(str))

# number of samples per each combined category
df_split['nbr_elapsed_days_target_sample_count'] = df_split.groupby('nbr_elapsed_days_target').no_show.transform('count')

# statistics of unique samples (single samples)
print('\'nbr_elapsed_days\' unique samples \'no_show\' mean:', round(df_split[df_split['nbr_elapsed_days_target_sample_count'] == 1].no_show.mean(),2),\
      '---- sample count:', df_split[df_split['nbr_elapsed_days_target_sample_count'] == 1].no_show.count())

# grouping unique samples into one category
df_split.loc[df_split.nbr_elapsed_days_target_sample_count == 1, "nbr_elapsed_days"] = 'rare'

# update count
df_split['nbr_elapsed_days_target_sample_count'] = df_split.groupby('nbr_elapsed_days').no_show.transform('count')

# drop temp column
df_split.drop('nbr_elapsed_days_target', axis = 1, inplace = True)

# update categorical and numeric columns
cat_cols = df_split.select_dtypes('object').columns
num_cols = df_split.columns[~df_split.columns.isin(cat_cols)]

# Sort column
no_show_idx = df_split.columns.get_loc("no_show")

columns_list = df_split.columns.tolist()

columns_sort = columns_list[0:no_show_idx] + columns_list[no_show_idx+1:] + columns_list[no_show_idx : no_show_idx+1]

df_split = df_split[columns_sort]

df_split.head()

In [599]:
# Add new feature to train and test set then target encode
X_train_mod = pd.concat([X_train_mod, df_split[df_split.index.isin(X_train_mod.index)][['nbr_elapsed_days']]], axis = 1, 
                    join= 'inner')

X_test_mod = pd.concat([X_test_mod, df_split[df_split.index.isin(X_test_mod.index)][['nbr_elapsed_days']]], axis = 1, 
                    join= 'inner')

# Target encode and transform test set
key = ['nbr_elapsed_days']

target_encode_feat_mapping(key, X_train_mod, X_test_mod, y_train, log_trans = True)

# Remove cat features, random forest works with only with numeric features
X_train_mod = X_train_mod.drop('nbr_elapsed_days', axis=1).copy()
X_test_mod = X_test_mod.drop('nbr_elapsed_days', axis=1).copy()

X_test_mod.head()

In [600]:
# worth it?

rf_results = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), 
                        X_train_mod, y_train, X_test_mod, y_test)

Not a worthy addition as we are interested in higher recall over precision.

In [601]:
rf_plot(rf_results.model, X_test_mod)

<li><a href="#toc">Table of contents</a></li>

<a id='fs'></a>

#### Feature selection

check [this notebook](https://www.kaggle.com/prashant111/comprehensive-guide-on-feature-selection) for various methods of feature selection with examples, we will be using univariate feature selection for simplicity.

In [602]:
# Feature selection
# code source--> https://github.com/abhishekkrthakur/approachingalmost

class UnivariateFeatureSelction:
    def __init__(self, n_features, problem_type, scoring, return_cols=True):
        """
        Custom univariate feature selection wrapper on
        different univariate feature selection models from
        scikit-learn.
        :param n_features: SelectPercentile if float else SelectKBest
        :param problem_type: classification or regression
        :param scoring: scoring function, string
        """
        self.n_features = n_features
        
        if problem_type == "classification":
            valid_scoring = {
                "f_classif": f_classif,
                "chi2": chi2,
                "mutual_info_classif": mutual_info_classif
            }
        else:
            valid_scoring = {
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression
            }
        if scoring not in valid_scoring:
            raise Exception("Invalid scoring function")
            
        if isinstance(n_features, int):
            self.selection = SelectKBest(
                valid_scoring[scoring],
                k=n_features
            )
        elif isinstance(n_features, float):
            self.selection = SelectPercentile(
                valid_scoring[scoring],
                percentile=int(n_features * 100)
            )
        else:
            raise Exception("Invalid type of feature")
    
    def fit(self, X, y):
        return self.selection.fit(X, y)
    
    def transform(self, X):
        return self.selection.transform(X)
    
    def fit_transform(self, X, y):
        return self.selection.fit_transform(X, y)
    
    def return_cols(self, X):
        if isinstance(self.n_features, int):
            mask = SelectKBest.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
                    
        elif isinstance(self.n_features, float):
            mask = SelectPercentile.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
        else:
            raise Exception("Invalid type of feature")
        
        return selected_features


ufs = UnivariateFeatureSelction(n_features = 20, problem_type = "classification", scoring = "mutual_info_classif")

features = X_train_mod.columns

ufs.fit(X_train_mod[features], y_train.values.ravel())

selected_features = ufs.return_cols(X_train_mod[features])

selected_features

In [603]:
# train on selected features only
X_train_mod_sf = X_train_mod[selected_features]
X_test_mod_sf = X_test_mod[selected_features]

rf_results = train_eval(RandomForestClassifier(max_depth=17, class_weight='balanced', random_state = 42), X_train_mod_sf, y_train, X_test_mod_sf, y_test)

<li><a href="#toc">Table of contents</a></li>

<a id='rts'></a>

#### Resampling training set

Classification results are affected by the unbalanced nature of the dataset, first thing we're going to do is to introduce more samples to both class labels using the data of patients having single appointment; next we're going to explore threshold moving and assesse total misclassification cost for each threshold.

In [604]:
# Resampling the training set by adding all sample of single patient appointments
# first we apply same preprocessing to single appointment dataframe

# avg elapsed days per neighborhood
df_first['avg_elaps_day_neighbourhood'] = df_first.groupby('neighbourhood').elapsed_days_sch.transform('mean').round()

# difference between elapsed day and average per neighborhood
df_first['elday_navg_diff'] = df_first.elapsed_days_sch - df_first.avg_elaps_day_neighbourhood

# Extracting additional timing features
df_first['sch_day'] =  pd.to_datetime(df_first.scheduling_day).dt.day
df_first['sch_month'] =  pd.to_datetime(df_first.scheduling_day).dt.month
df_first['sch_hour'] =  pd.to_datetime(df_first.scheduling_day).dt.hour
df_first['app_month'] =  pd.to_datetime(df_first.appointment_day).dt.month
df_first['day_of_year_sch'] =  pd.to_datetime(df_first.scheduling_day).dt.dayofyear
df_first['time_stamp_sch'] = pd.to_datetime(df_first.scheduling_day).dt.time.astype('str')

# Remove seconds from time stamp
df_first['sch_time'] = df_first.time_stamp_sch.apply(lambda x: str(x)[:-3]).astype('str')

# filler columns for merging, calculations will be updated later
df_first['msd_app_sch_slot_nb'] = 0
df_first['msd_app_sch_slot'] = 0

# Mark all samples to easily identify later
df_first['n'] = 1

df_first.head(2)

In [605]:
# Preprocessing function

def combine_group(df, features, target, label_encode = False):
    ''' Combine unique samples into unique levels and apply label encoding '''
    
    # incase several features are passed at once
    if isinstance(features, list):
        features = features
    else:
        features = [features]
    
    for f in features:
        
        # Combining with target to check sample count
        df[f'{f}_target'] = (df[f].astype(str) + '_' + df[target].astype(str))

        # combined feature sample count
        df[f'{f}_sample_count'] = df.groupby(df[f'{f}_target'])[target].transform('count')
        
        # grouping unique samples under one category
        if df[df[f'{f}_sample_count'] == 1][f'{f}_sample_count'].sum() > 0:
            
            df.loc[df[f'{f}_sample_count'] == 1, f"{f}"] = 'rare'
            
            # statistics of unique samples (single samples)
            print(f'\'{f}\' unique samples \'no_show\' mean:', round(df[df[f'{f}_sample_count'] == 1].no_show.mean(),2),\
                  '---- sample count:', df[df[f'{f}_sample_count'] == 1].no_show.count())
        
            # update count
            df[f'{f}_sample_count'] = df.groupby(df[f'{f}'])[target].transform('count')
            
            # drop temp column
            df.drop(f'{f}_target', axis = 1, inplace = True)
        
        if label_encode:    
            encoder = LabelEncoder()
            df[f'{f}_'] = encoder.fit_transform(df[f'{f}'])

In [606]:
# Starting Fresh - Resampling then apply target encoding

# match columns of the two dataframes as single appointment patients do not have all features of those with multiple appointments
df_split_re = df_split[df_first.columns[:-1]].copy()

# Mark old samples
df_split_re['n'] = 0

# Combine the two dataframes
df_split_re = df_split_re.append(df_first, ignore_index=True)

# Update numeric features by new samples
df_split_re['msd_app_sch_slot_nb'] = df_split_re.sort_values(['appointment_day','scheduling_day']).\
groupby(['neighbourhood', 'sch_day', 'sch_month', 'sch_hour']).no_show.shift().fillna(0)

df_split_re['msd_app_sch_slot_nb'] = df_split_re.sort_values(['appointment_day','scheduling_day']).\
groupby(['neighbourhood', 'sch_day', 'sch_month', 'sch_hour']).msd_app_sch_slot_nb.cumsum().fillna(0)

df_split_re['msd_app_sch_slot'] = df_split_re.sort_values(['appointment_day', 'scheduling_day']).groupby(['sch_day', 'sch_month', 'sch_hour'])\
.no_show.shift().fillna(0)

df_split_re['msd_app_sch_slot'] = df_split_re.sort_values(['appointment_day', 'scheduling_day']).groupby(['sch_day', 'sch_month', 'sch_hour'])\
.msd_app_sch_slot.cumsum().fillna(0)

# Remove previous grouping
df_split_re['time_stamp_sch'] = pd.to_datetime(df_split_re.scheduling_day).dt.time.astype('str')
df_split_re['sch_time'] = df_split_re.time_stamp_sch.apply(lambda x: str(x)[:-3]).astype('str')
df_split_re["nbr_sch_time"] = (df_split_re.neighbourhood + "_" + df_split_re.sch_time.astype(str))
df_split_re["nbr_elapsed_days"] = (df_split_re.neighbourhood + "_" + df_split_re.elapsed_days_sch.astype('str'))

# Group unique samples
features = ['time_stamp_sch', 'sch_time', 'neighbourhood', 'nbr_sch_time', 'nbr_elapsed_days']

target = 'no_show'

combine_group(df_split_re, features, target, label_encode = True)

# Sort column
no_show_idx = df_split_re.columns.get_loc('no_show')

columns_list = df_split_re.columns.tolist()

columns_sort = columns_list[0:no_show_idx] + columns_list[no_show_idx+1:] + columns_list[no_show_idx : no_show_idx+1]

df_split_re = df_split_re[columns_sort]

df_split_re.head()

In [607]:
# Train/Test split, we have different feature setup than previous split as now we incorporated patients with single appointments
# they miss some of the features created for the ones having multiple appointments

# separate categorical and numeric columns
cat_cols_re = df_split_re.select_dtypes('object').columns
num_cols_re = df_split_re.columns[~df_split_re.columns.isin(cat_cols_re)]

# remove unnecessary cat cols
mask = df_split_re.drop(cat_cols_re[0:5], axis=1)

# unifying numeric dtypes
mask[num_cols_re[:-1]] = mask[num_cols_re[:-1]].astype('float', errors='ignore')

# split old samples, add all new samples to train set later
mask_old = mask[mask.n == 0].copy()

# split data
features_re = mask_old.drop(['time_stamp_sch_sample_count', 'sch_time_sample_count', 
                             'neighbourhood_sample_count', 'nbr_sch_time_sample_count', 
                             'nbr_elapsed_days_sample_count', 'n', 'no_show'], axis=1)
labels_re = mask_old.no_show

X_train_re, X_test_re, y_train_re, y_test_re = train_test_split(features_re, labels_re, stratify = labels_re, 
                                                                test_size=.2, random_state=42)

X_train_re.head()

In [608]:
# no_show statistics before resampling
y_train.value_counts(), y_train.value_counts(normalize=True).round(2)

In [609]:
# Resample train set

# Copies to modify
X_train_mod = X_train_re.copy()
X_test_mod = X_test_re.copy()

# using both +ve and -ve examples
X_train_mod = X_train_mod.append(df_split_re[(df_split_re.n == 1)][features_re.columns])

# update labels
y_train_re = y_train_re.append(df_split_re[(df_split_re.n == 1)]['no_show'])

y_train_re.value_counts(), y_train_re.value_counts(normalize=True)

In [610]:
# Target Encode

# cols to encode
key = ['time_stamp_sch', 'sch_time', 'neighbourhood', 'nbr_sch_time', 'nbr_elapsed_days']

target_encode_feat_mapping(key, X_train_mod, X_test_mod, y_train_re, log_trans = True)

# Remove cat features features
X_train_mod_re = X_train_mod.drop(cat_cols_re[5:], axis=1).copy()
X_test_mod_re = X_test_mod.drop(cat_cols_re[5:], axis=1).copy()

X_test_mod_re.head()

<a id='prevr'></a>

In [611]:
# worth it?

rf_results_re = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), X_train_mod_re, y_train_re, 
                                                                                               X_test_mod_re, y_test)

Resampling the training set resulted in better classification performance compared to previous results before resampling. 

Note that we are using both label and target encoded cat features and removed 'nbr_sch_stamp' feature completely; however, performance is still improved after adding new samples even without these modifications.

In [612]:
rf_plot(rf_results_re.model, X_test_mod_re)

<li><a href="#impr">Fast forward to improved results</a></li>

In [613]:
# Feature selection

ufs = UnivariateFeatureSelction(n_features = 20, problem_type = "classification", scoring = "mutual_info_classif")

features = X_train_mod_re.columns

ufs.fit(X_train_mod_re[features], y_train_re.values.ravel())

selected_features = ufs.return_cols(X_train_mod_re[features])

selected_features

<a id='prevr2'></a>

In [614]:
# Train on selected features only
X_train_mod_re_sf = X_train_mod_re[selected_features]
X_test_mod_re_sf = X_test_mod_re[selected_features]

rf_results_re_sf = train_eval(RandomForestClassifier(min_samples_leaf = 5, class_weight='balanced', random_state = 42), 
                              X_train_mod_re_sf, y_train_re, X_test_mod_re_sf, y_test)

Same consistent improved performance compared to previous predictions on selected features but before resampling the training set. As a reminder, I'm seeking a higher recall on +ve class without significant loss in precision as there will always be a tradeoff between these two metrics and predicting a false 'no show' (false positive) is better than a false show (false negative) hypothetically; we will discuss this in more details latter when selecting a threshold for classification.

Note that we've been addressing class imbalance issue so far on the classifier level by setting 'class_weight' hyperparameter to 'balanced', this will force the classifier to pay equal attention to both classes; one way to set the class weight parameter is to inverse the current distribution of class labels (‘class_weight’ = {0:.8, 1:.2}) or play around with some other percentages till you achieve desired results.

Additionally we've increase the required number of samples to consider a node to be a leaf node (no further splitting) from 1 (default) to 5 and this is to prune trees and reduce overfitting.

Pruning? Overfitting? .....

<li><a href="#impr2">Fast forward to improved results</a></li>

In [615]:
# Sample of a decision Tree

# grow a tree
med_tree = tree.DecisionTreeClassifier(max_depth = 3)
med_tree.fit(X_train_mod_re_sf, y_train_re)

# Plot
fig  = plt.subplots(figsize=(11, 11), dpi=800)
plt.tight_layout()

tree.plot_tree(med_tree, filled=True, rounded=True, feature_names =  X_train_mod_re_sf.columns, fontsize=9);

Tree classifiers cast a series of yes/no questions accordingly classification routes are determined, the depth of tree indicates the number of decisions that have to be made down the longest path through the tree. Important features for decision making are used as a splitting points early on (or near the top of the tree), that’s why 'time_stamp_sch_' is being used as a first decision split point, we know from random forest feature importance plots that this feature is the one with highest importance score.

Boxes are called nodes; there are two types of nodes: 1) Split nodes that pose yes/no question and leads to further splitting; the very first split node is called root node. 2) Terminal nodes (leaf nodes) is where classifications are made. In the plot above, the terminal nodes(leaf nodes) are the nodes at the bottom of the figure that have no branches or further decision nodes below them(no further splitting). 

Each arrow represents an answer to the posed question, arrows to the left hand side (right side of the root node pointing to 'time_stamp_sch_score') are the 'Yes' answer to the condition within the box; opposite arrows are the 'No' answer.

Node colors indicate class membership, where brownish colors represent -ve class and bluish colors represents +ve class. The intensity of coloring follows class concentration within each node, the higher the number of samples of a given class the darker the color.

Text within each node holds several information about the structure of the node, such as:

- Split criteria, which is the basis for making a split decision
- Measure of split quality based on Gini and Entropy criteria. The choice does not make a lot of difference in performance.
- Total sample size
- Class distribution per node, root node holds all samples and same class distribution as seen the training set since no decision is being made.

How Predictions are made?

Split point is first determined, and then each sample in the dataset is passed to a non-terminal node where further splitting takes place until tree structure is achieved. 

Now let’s follow a decision route from our sample tree, the one at the right hand side leading to the blue leaf node:
- time_stamp_sch_ <=0.21, No, nbr_sch_time_score <= 0.18, No, nbr_elapsed_days_score <= 0.18, No, then all samples within this criteria gets classified as a +ve class.

The structure of our sample tree is of depth 3 and minimum sample per leaf is 1, this means that the tree is only allowed a total of 3 splits before reaching to a classification decision, and each split will only take place if the resulting leaf nod holds at least 1 sample. Such structure may not be sufficient depending on the problem at hand, sometimes deep trees are needed and best thing to do is to test multiple ranges of values and their impact on classification results.

Pruning is controlling the tree growth in a way that maintains a balance between the amount of information needed to produce accurate predictions and at the same time control overfitting.

<li><a href="#toc">Table of contents</a></li>

<a id='ovrft'></a>

#### Overfitting

Simply put, overfitting is when the classifier learns the training data by heart and generates excellent predictions results for that data but can't reproduce such performance on the test/validation data; this is usually referred to as poor generalization performance.
 
Why does this happen?
- Unbalanced and unrepresentative training data
- Too many features to learn from
- Un-tuned classifier
- Bad features that does not represent the underlying problem

Why is it problematic?
- Degraded performance on unseen data
- Wasted efforts, the time and resources spent on building a model is not fully utilized because of poor generalization performance.

How to handle overfitting?
- First be conscious that the model is overfitting
- Cross-Validate
- Hyperparameter tuning, pruning and regularization.
- Revisit approach
- Feature selection/ limiting number of features available for training
- Maintain a balanced and representative training set

The above are not exhaustive lists of all reasons of overfitting or counteracts because it really depends on the problem at hand. Should we be worried that the model overfits? Yes, but in the context of how well it performs on unseen data. Generalization is key measure of model performance, we should be aiming to produce a model with as much generalizability as possible and be conscious about when the model overfits and try to reduce its impact; trying to avoid over-fitting completely might lead to under-fitting, where we regularize too much and fail to learn relevant information. 

We are not going to care much about overfitting issue in this project, but it had to be mentioned as a sub-topic in modeling because of its importance; next we going to explore the effect of overfitting on both training and test results and how some of the earlier mentioned counteracts can help control overfitting specifically pruning and maintaining a representative training set.

In [616]:
# analysing overfitting effect based on tree depth
max_d = [5,10,14,18]

for d in max_d:
    # model
    rf = RandomForestClassifier(max_depth = d, class_weight='balanced', random_state = 42)
    # fit 
    rf.fit(X_train_mod_re_sf, y_train_re)
    # PR_AUC of train-set
    train_pred = rf.predict_proba(X_train_mod_re_sf)    
    pr_precision, pr_recall, pr_thresh = precision_recall_curve(y_train_re, train_pred[:,1])
    pr_auc_train = auc(pr_recall, pr_precision)
    # PR_AUC of test-set
    y_pred = rf.predict_proba(X_test_mod_re_sf)
    pr_precision, pr_recall, pr_thresh = precision_recall_curve(y_test_re, y_pred[:,1])
    pr_auc_test = auc(pr_recall, pr_precision)
    print(f'Precision Recall AUC is {pr_auc_train:.0%} and {pr_auc_test:.0%} for train and test set respectively at max_depth of {d}')

We are computing the PR AUC score for predictions of training and testing set separately at different pruning level, setting the max depth of decision tree is a form of pruning which aims at controlling overfitting.

First thing to notice is the gap between AUC score for both training and testing sets the less trees the lower the gap, another thing to note is that increasing the number of trees does in fact increase AUC scores for both sets but up to a certain limit; there is no benefit of increasing the max depth beyond 10 as the training score improves while testing score stalls and at the same time if max depth was set to 5 we would be missing on some improvement in prediction results gained by these additional trees despite the fact that it overfits.

Just remember that *'If it fits, it overfits’!*

<li><a href="#toc">Table of contents</a></li>

<a id='thrm'></a>

#### Threshold Moving

One way to address class imbalance issue even before attempting to rebalance/resample the training set is Threshold Moving. Some classifiers, including decision trees, are capable of providing two types of prediction results being class label [0,1] and/or the probability of class membership. 

When predicting class membership probability, classification results are displayed in the form of normalized probabilities for each class label, so let’s assume that the predicted probabilities for an unseen sample are [0.3,0.7] this means that there is 30% chance that this sample belong to 1st class, and 70& chance it belongs to the other class being 0 and 1 respectively; for simplicity we reduce this representation to focus only on the +ve class. Going back to our example we would be interested in the [0.7] part of prediction results.

To convert theses probabilities to class labels [0,1] there need to be some sort of guidance for mapping prediction probabilities to class labels, this is known as classification threshold with a default value of 0.5. This means that any sample in the test set that has predicted probability equal to or greater than 0.5 will be classified as a member of the class to which the prediction probability refer to, this may not be optimal and we can control this by threshold moving. 

Threshold moving is simply changing the default classification threshold to whatever works best for problem at hand, going back to our example let assume that the predicted probabilities are [0.6,0.4] which means that this sample belongs to the -ve class since the probability of being a member of +ve class is only 0.4 and the default class assignment threshold is 0.5; by setting the classification threshold to 0.4 this sample will be classified as a member of the +ve class.

But what threshold to use and why?

In [617]:
# Threshold moving - Calculating and plotting Area under the cure for Precision recall and ROC curve

# Precision Recall Curve
fig  = plt.subplots(figsize=(10, 4))
plt.tight_layout()

plt.subplot(1, 2, 1) # 2 rows , 2 col , index 1

# calculate f1_score for each threshold
fscore_rf = (2 * rf_results_re_sf.pr_precision * rf_results_re_sf.pr_recall) / (rf_results_re_sf.pr_precision + rf_results_re_sf.pr_recall)

# optimal(largest) f1_score threshold location on plot
ix_f_rf = np.argmax(fscore_rf)

# plot the precision-recall curves
no_skill = len(y_test[y_test==1]) / len(y_test)
plt.plot([0, 1], [no_skill, no_skill], 'r--')
plt.plot(rf_results_re_sf.pr_recall, rf_results_re_sf.pr_precision, 'b', label = 'AUC = %0.2f' % rf_results_re_sf.pr_auc, linewidth=2)
plt.scatter(rf_results_re_sf.pr_recall[ix_f_rf], rf_results_re_sf.pr_precision[ix_f_rf], marker='o', color='red', label='Best Threshold')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title("PR_AUC RF - Medical Appointments")
plt.legend(loc = 6)
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

plt.subplot(1, 2, 2)

# ROC Curve
# calculate the g-mean for each threshold
j_rf = rf_results_re_sf.tpr - rf_results_re_sf.fpr

# optimal(largest) g-mean threshold location on plot
ix_j_rf = np.argmax(j_rf)

plt.plot(rf_results_re_sf.fpr, rf_results_re_sf.tpr, 'b', label = 'AUC = %0.2f' % rf_results_re_sf.roc_auc, linewidth=2)
plt.plot([0,1], [0,1], 'r--')
plt.scatter(rf_results_re_sf.fpr[ix_j_rf], rf_results_re_sf.tpr[ix_j_rf], marker='o', color='red', label='Best Threshold')
plt.legend(loc='lower right')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC_AUC RF - Medical Appointments")
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0);

First step in threshold moving is to select a threshold, we may have already a set of known thresholds that we would like to test or we can select a threshold that optimize a given performance measure. The above two plots represents AUC for PR and ROC curves calculated at different classification thresholds of random forest trained on selected features, we note that AUC is 71 and 91 for PR and ROC curve respectively and these are the results of earlier predictions based on 0.5 classification threshold.

The red dots correspond to location of optimal classification threshold that maximizes a given performance measure, F-measure (f1 score) is used as the performance measure optimization for PR Curve while Geometric mean is chosen for ROC Curve. The Geometric Mean or G-Mean is a metric for imbalanced classification that, if optimized, will seek a balance between the true positive rate (tpr) and the false positive rate (fpr). G-Mean = sqrt(tpr * (1-fpr)) or (tpr - fpr) for faster calculation according to [Youden’s J statistic](https://en.wikipedia.org/wiki/Youden%27s_J_statistic)

We are particularly interested in PR Curve because unlike the ROC Curve, it focuses on the performance of a classifier on the positive (minority class) only and we can observe this in the differences between each curve's AUC scores being very high for ROC curve and lower for PR curve, reason being imbalanced nature of dataset where ROC metrics (tpr and fpr) are shadowed by the high number of samples for the -ve examples and the classifiers' performance on those examples compared to +ve ones.

If we are interested in a threshold that results in the best balance of precision and recall, then this is the same as optimizing the F-measure that summarizes the harmonic mean of both measures. Selecting threshold that maximize f1 score is not always the best choice cost wise, also that best balance between precision and recall sometimes is not required as the cost of low recall outweigh that of low precision.

Each point on these curves corresponds to a different classification threshold and a separate confusion matrix, knowing associated costs of misclassification errors can help us make guided decision on which threshold to choose; we just have to choose the confusion matrix that would imply the cost matrix we are looking. We might also want to test more thresholds apart from those on the curves.

In [618]:
# Threshold moving - Calculating costs of classification errors at different decision threshold 

# choosing a cut-off threshold
# here we are selecting the earlier calculated optimum threshold that maximize F-measure and some others lower/higher
# thresholds to compare results
cutoff = [rf_results_re_sf.pr_thresh[ix_f_rf - 1000], rf_results_re_sf.pr_thresh[ix_f_rf - 800], rf_results_re_sf.pr_thresh[ix_f_rf - 600], 
          rf_results_re_sf.pr_thresh[ix_f_rf - 400], rf_results_re_sf.pr_thresh[ix_f_rf], rf_results_re_sf.pr_thresh[ix_f_rf + 400], 
          rf_results_re_sf.pr_thresh[ix_f_rf + 600], rf_results_re_sf.pr_thresh[ix_f_rf + 800], rf_results_re_sf.pr_thresh[ix_f_rf + 1000]]

# Instantiate empty list to hold performance metrics of different classification threshold
misclss_err = []

for c in cutoff:
    
    # Converting probabilities to class labels using new threshold
    y_pred_classes = np.zeros_like(rf_results_re_sf.pred_p)             # initialize a matrix full with zeros
    y_pred_classes[np.array(rf_results_re_sf.pred_p) > c] = 1           # add a 1 if the cutoff was breached
    
    # Calculate performance metrics for the a given threshold
    pr_precision, pr_recall, _ = precision_recall_curve(y_test, y_pred_classes)
    pr_f1, pr_auc = f1_score(y_test, y_pred_classes), auc(pr_recall, pr_precision)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_classes)
    roc_auc = auc(fpr, tpr)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_classes).ravel()
    err_rate = (fp+fn) / len(y_pred_classes)
    
    # update list
    misclss_err.append((tn, fp, fn, tp, err_rate.round(4), pr_precision[1].round(3), pr_recall[1].round(3), pr_f1.round(3), pr_auc.round(3), 
                        roc_auc.round(3), c.round(5) , y_pred_classes))

# Display results of each classification threshold
misclss_err_df = pd.DataFrame([i[0:11] for i in misclss_err], columns=['tn', 'fp', 'fn', 'tp', 'err_rate', 'precision', 'recall', 'f1-score', 'pr_auc', 'roc_auc',
                                                                       'threshold'])    

# assuming different cost values for tp / fn
misclss_err_df['fp_cost'], misclss_err_df['fn_cost'] = misclss_err_df.fp * 10, misclss_err_df.fn * 100
misclss_err_df['total_misclss_cost'] = misclss_err_df.fp_cost + misclss_err_df.fn_cost

misclss_err_df

In [619]:
# Threshold that maximize f-score
rf_results_re_sf.pr_thresh[ix_f_rf]

Second step is to assess classification performance at the selected thresholds; here we've made such assessment using different classification thresholds, including the one that maximize our selected performance measure (F-measure).

The above table displays performance metrics at different classification thresholds, along with associated hypothetical cost of total misclassification errors that is equal to false positive and false negative predictions multiplied by corresponding monetary value. We've established the monetary value of false negative predictions (predictions = show, actual = ‘no show’) to be 10 times more than those of false positive ones (predictions = ‘no show’, actual = show) reason being that usually the costs associated with false negative predictions in imbalanced classification problems outweigh that of false positive ones.

Classification thresholds are sorted in an ascending order with first row (index 0) displaying results of very low threshold, a conservative approach where we classify everything above 27% as being a ‘no show’; this result in a higher recall of +ve class and the lowest misclassification error cost among the selected thresholds. The 5th row (index 4) displays results of the classification threshold that maximize F-measure (f1 score). 

We can note that misclassification error hypothetical costs tends to be higher at higher classification thresholds due to the fact that recall of +ve class is lower and vice versa; similarly is error rate its decreasing due to difference in sample size of each class and the imbalanced nature of the dataset but collectively the decline have an inverse effect on total misclassification cost given the significance of one class (+ve) over the other (-ve).

So which threshold to choose? It really depends on what we want to achieve with our classification task, costs associated with misclassification errors can be a starting point to compare different thresholds. Regardless of the threshold we will choose and for this approach to work properly, we’ll need to get a proper estimate of the costs associated with each type of misclassification error and we’ll need to make sure that the percentages of positive cases and negative cases match those the model will see at deployment.

Summary:

- Associated costs of different confusion matrix need to be accounted for when assessing model performance
- Sometimes having high recall on +ve class outweigh any other performance measure.
- There are many techniques that may be used to address an imbalanced classification problem; perhaps the simplest approach is to change the decision threshold.

<li><a href="#toc">Table of contents</a></li>

<a id='kfcv'></a>

#### K-fold cross validation

So far we've been testing the model using fixed training/testing sets while addressing class imbalance issue on both classifier level (class_weight hyperparameter) and data level (collecting more sample from both classes).

Next we explore k-fold cross validation where the model is tested using different train/test sets.

In [620]:
# Cross validation - Instantiating folds

# Copies to modify
X_train_mod_re_cv = X_train_mod_re.copy()
X_test_mod_re_cv = X_test_mod_re.copy()

# Reattach target for CV split
X_train_mod_re_cv['target'] = y_train_re

# k-fold splits
X_train_mod_re_cv["kfold"] = -1

X_train_mod_re_cv = X_train_mod_re_cv.sample(frac=1).reset_index(drop=True)

y_train_re_cv = X_train_mod_re_cv.target

kf = StratifiedKFold()

for f, (t_, v_) in enumerate(kf.split(X = X_train_mod_re_cv, y = y_train_re_cv)):
    X_train_mod_re_cv.loc[v_, 'kfold'] = f
    
X_train_mod_re_cv.head()

In [621]:
# Class distribution per each fold
X_train_mod_re_cv.groupby('kfold').target.value_counts(normalize=True)

Now we test model based on cross validation folds. For fold 0 the model is trained on folds 1-4 and tested on fold 0, the process is repeated in the same manner across the 5 folds/iterations (fold 1 train model on fold 0 and 2-4 then test on fold 1, and so on).

We using stratified k-fold as the data set is imbalanced and we want to have equal representation of both classes in each fold.

In [622]:
# Cross validation - Train and evaluate

def train_eval_cv(fold, df, model, eval_func):
    # training data is where kfold is not equal to provided fold
    df_train = df[df.kfold != fold].reset_index(drop=True)
    
    # validation data is where kfold is equal to provided fold
    df_test = df[df.kfold == fold].reset_index(drop=True)
    
    # instantiate train/test features/labels
    x_train = df_train.drop(['target', 'kfold'], axis=1)
    y_train = df_train.target
    
    x_test = df_test.drop(['target', 'kfold'], axis=1)
    y_test = df_test.target
    
    # evaluate model
    eval_func(model, x_train, y_train, x_test, y_test)

# Model evaluation CV
fold = [0,1,2,3,4]

model = RandomForestClassifier(class_weight='balanced', random_state = 42)

for f in fold:
    print('-' * 40, '\n', f'Fold {f + 1} results:', '\n', '-' * 40)
    train_eval_cv(f, X_train_mod_re_cv, model, train_eval)
    print('\n', '-' * 40, '\n')

How come CV results are better than fixed train/test set? Because we have leakage!

We've instantiated folds after target encoding so train sets within folds 'knows' about target labels of test sets, let’s fix this!

In [623]:
# Target encoding within Cross validation folds

def train_eval_cv(fold, df, model, key, target_encode_func, eval_func, target = None,
                  custom_encoding = True, log_trans = False, with_weight = True, drop_cat_feats = False, cat_cols = None):
  
    # training data is where kfold is not equal to provided fold
    df_train = df[df.kfold != fold].reset_index(drop=True)
    
    # validation data is where kfold is equal to provided fold
    df_test = df[df.kfold == fold].reset_index(drop=True)
    
    # instantiate train/test features/labels
    x_train = df_train.drop([target, 'kfold'], axis=1)
    y_train = df_train[target]
    
    x_test = df_test.drop([target, 'kfold'], axis=1)
    y_test = df_test[target]
    
    # Target Encode
    if custom_encoding:
        target_encode_func(key, x_train, x_test, y_train, log_trans = log_trans, with_weight = with_weight)
    else:
        target_encode_func.fit(x_train, y_train)
        x_train = target_encode_func.transform(x_train)
        x_test = target_encode_func.transform(x_test)
        
    if drop_cat_feats:
        # Remove cat features
        x_train = x_train.drop(cat_cols, axis = 1).copy()
        x_test = x_test.drop(cat_cols, axis = 1).copy()

    # evaluate model
    eval_func(model, x_train, y_train, x_test, y_test)

In [624]:
# Instantiate folds

# copies to modify
df_split_re_cv = df_split_re.copy()

# remove unnecessary cols
df_split_re_cv = df_split_re_cv.drop(['patient_id', 'appointment_id', 'gender', 'scheduling_day', 'appointment_day', 
                                      'time_stamp_sch_sample_count', 
                                      'sch_time_sample_count', 'neighbourhood_sample_count', 
                                      'nbr_sch_time_sample_count', 'nbr_elapsed_days_sample_count', 
                                      'n'], axis=1)

# k-fold splits
df_split_re_cv["kfold"] = -1

df_split_re_cv = df_split_re_cv.sample(frac=1).reset_index(drop=True)

y_train_cv = df_split_re_cv.no_show

kf = StratifiedKFold()

for f, (t_, v_) in enumerate(kf.split(X = df_split_re_cv, y = y_train_cv)):
    df_split_re_cv.loc[v_, 'kfold'] = f
    
df_split_re_cv.head()

In [625]:
# Class distribution per fold
df_split_re_cv.groupby('kfold').no_show.value_counts(normalize=True)

In [628]:
# Model evaluation k-fold cross validation
fold = [0,1,2,3,4]

encoder = target_encode_feat_mapping

# # play around with different encoders
# encoder = ce.TargetEncoder(min_samples_leaf = 10)

model = RandomForestClassifier(random_state = 42)

key = ['time_stamp_sch', 'sch_time', 'neighbourhood', 'nbr_sch_time', 'nbr_elapsed_days']

cat_cols = cat_cols_re[5:10]

for f in fold:
    print('-' * 40, '\n', f'Fold {f + 1} results:', '\n', '-' * 40)
    train_eval_cv(f, df_split_re_cv, model, key, encoder, train_eval, target = 'no_show',
                  custom_encoding = True, log_trans = True, drop_cat_feats = True, cat_cols = cat_cols)
    print('\n', '-' * 40, '\n')

Target encoding at each folds now portrays less optimistic performance than when target encoding was done wrongfully before splits. Overall, K-fold cross validation using full data displays consistent performance across folds similar to what we've seen earlier on fixed train/test split using all features.

Note that we are applying target encoding at each fold but following the same principles we used when applying at fixed train/test split; this is not to be confused with K-fold target encoding which is a different method of target encoding aiming at reducing overfitting where target scores of each fold are derived from the other folds.

Usually it's better to use k-fold cross validation than a fixed train/test set to gain some comfort on the model's generalized performance, another way to assess generalization performance is to visually examine classification decision boundaries.

But what is a Decision Boundary?

<li><a href="#toc">Table of contents</a></li>

<a id='vdb'></a>

#### Visualizing Decision Boundary

In [629]:
# plotting samples of train data points

# full data
X = X_train_mod_re_sf.append(X_test_mod_re_sf)
y = y_train_re.append(y_test_re)

# Scale data for PCA
# scaler = StandardScaler()
# scaler = RobustScaler()
scaler = PowerTransformer()

X_scaled = pd.DataFrame(scaler.fit_transform(X), index = X.index, columns = X.columns)

# PCA for 2D plotting
pca = PCA(n_components=2, random_state = 45)
Xreduced = pca.fit_transform(X_scaled)

print('Explained Variance Ratio for PC1 and PC2 respectively', np.round(pca.explained_variance_ratio_, 2))

pca_df_vis = pd.DataFrame(np.column_stack((Xreduced, y)), columns=['PC1', 'PC2', 'no_show'], index = X_scaled.index)

def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, model = '', **params):
    if model == 'svm':
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

# Set-up grid for plotting.
X0, X1 = Xreduced[:, 0], Xreduced[:, 1]
xx, yy = make_meshgrid(X0, X1)

# Plot
fig, ax1= plt.subplots(figsize=(20, 10))

# RandomForest
model = RandomForestClassifier(max_depth = 15, class_weight='balanced', random_state = 42)

# train model using same data used in latest model as we are visualising the boundaries of that model 
clf = model.fit(pca_df_vis[pca_df_vis.index.isin(X_train_mod_re_sf.index)][['PC1', 'PC2']].values, 
                y[y.index.isin(y_train_re.index)])

# plot mesh grid
plot_contours(ax1, clf, xx, yy, cmap='RdYlBu', alpha=0.8)

# plot sample of train data
train_sample = pca_df_vis[pca_df_vis.index.isin(X_train_mod_re_sf.index)].sample(1000, random_state = 42).sort_index()
labels_tr_sample = y[y.index.isin(train_sample.index)].sort_index()

_ = ax1.scatter(train_sample.PC1, train_sample.PC2, c =  labels_tr_sample , s = 80,  edgecolors = 'k', cmap = 'RdYlBu')

# # plot +ve samples only
# train_sample = pca_df_vis[(pca_df_vis.index.isin(X_train_mod_re_sf.index)) & 
#                           (pca_df_vis.no_show == 1)].sort_index()
# labels_tr_sample = y[y.index.isin(train_sample.index)].sort_index()

# _ = ax1.scatter(train_sample.PC1, train_sample.PC2, c =  'blue' , s = 80,  edgecolors = 'k')

# Decoration
ax1.set_ylabel('PC2'); ax1.set_xlabel('PC1')
ax1.set_title('Random Forest - Decision boundary', fontsize = 15)

plt.legend(handles = _.legend_elements()[0], labels = ['Show', 'No_show'], loc=4, fontsize = 'large')
plt.tight_layout();

What is a decision boundary/surface?

It's the assignment of class labels along the feature space, color shades represent predicted probabilities of class membership for a given sample where reddish regions mark -ve class labels and bluish regions mark the +ve ones. Color intensity signals the classifier's confidence in class membership predictions for given sample; the darker the higher probability of being a member of the class for which the color represents.

In order to create this plot we draw a meshgrid (background) of prediction probabilities then we plot the desired train/test data points to know where each points fits and how the classifier 'sees' them; in the above plot we are plotting only a sample of training data points to have a less crowded plot and we are using PCA for visualizing the feature space in a 2D plot. Note that these 2 PCs represent only 35% of the variance but we can conclude the following:

- We still have overlapping issues, despite the fact that 'no show' appointments tends to be more at PC1 > 0. It should be apparent that because of this overlap, any decision boundary will necessarily misclassify some observations fairly often and this is what we actually saw in classification results having fairly large amount of false positive/negative classifications.

- Decision boundary is fairly complicated with several signs of overfitting; we are using trees of depth 15 and can observe carved decision boundaries around some data point. The color shades of decision boundaries follow the respective class label intensity in any given region; however, complicated boundaries usually do not generalize well on unseen data resulting in degraded performance. Complexity of the model depends on the number of features used and the level of regularization/pruning applied, there is no ‘best’ measurement of complexity as it varies from one problem to another but in general we should aim for a less complicated model that generalize well on unseen data


We'll next look at the decision boundaries of samples from test data and examine the characteristics of adjacent and disjoint data points to understand what distinguish these appointments from one another and what should be our next steps.

In [630]:
# Plotly interactive visualization in case we want to inspect individual data points
# we now plot samples of test data points

# meshgrid
h = .02  # step size in the mesh
X = pca_df_vis.values # full data

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

y_ = np.arange(y_min, y_max, h)

# contour RF
fig = subplots.make_subplots(rows=1, cols=1)

pred = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
pred = pred.reshape(xx.shape)

t1 = go.Heatmap(x = xx[0], y = y_, z = pred, colorscale='RdYlBu', showscale=False)

# plot sample of test data
test_sample = pca_df_vis[pca_df_vis.index.isin(X_test_mod_re_sf.index)].sample(1000, random_state = 42).sort_index()
labels_ts_sample = y[y.index.isin(test_sample.index)].sort_index()

t2 = go.Scatter(x = test_sample.PC1,
                y = test_sample.PC2,
                mode = 'markers', showlegend = False, marker = dict(size=10, color = labels_ts_sample, colorscale = 'RdYlBu', 
                                                                    line = dict(color = 'black', width = 1)),
                hovertemplate = '<br>X</b>: %{x}' + '<br>Y</b>: %{y}<br>'+ '<br>Index: %{text}', text = test_sample.index)
                   
                
fig.append_trace(t1, 1, 1)
fig.append_trace(t2, 1, 1)
fig.update_layout(autosize=False, width=1200, height=700)

In [633]:
# full features, co-ords (x:-2.4, y:2.9 and x:-2.1, y=3.3) for first two samples and (x:-4.2, y:4.7) for the last one
df_split_re.loc[[39621, 3562, 43673]]

In [634]:
# features used to train the model
X_test_mod_re_sf.loc[[39621, 3562, 43673]]

In [635]:
# prediction results
predictions_df.loc[[39621, 3562, 43673]][['no_show', 'pred_p', 'pred_c']]

First two samples are adjacent and we can observe that most features used in training are somewhat similar except for the target scores which are also similar for the highly discriminating features used by the model for making predictions ('time_stamp_sch'). The last sample is clearly distinct from the first two in most of the features and this justifies its location in the plot from these points, as for predictions the +ve class sample of the first two examples is wrongly classified and this is caused by mainly the following:

- Similarity of features for both examples
- Class imbalance as this region of the plot is dominated by -ve samples, imbalance issue is also reflected in the levels per categorical features.
- Scores of highly discriminating features do not portray the fact that they belong to different criteria contrary to what they were designed to do, we are using the exact time stamps captured in 'time_stamp_sch' feature as a mean of highlighting a common criteria belonging to these specific timing (nature of appointment, venue, physician, etc..) that we do not know about but we believe that it has a direct cause of 'no show'; the way we are grouping unique samples currently does not serve this purpose well.

Given the above, a lot of things that can be done come to one’s mind such as:
- Redesign training set instead of random splits
- Under/oversample training set
- Use only +ve class samples of single appointment patients when resampling the training set
- Remove some of the -ve class samples from each categorical feature level
- Change categories grouping scheme
- Change weight calculation in custom target encoding
- Other method of feature selection
- Grid search hyperparameter tuning
- Calibrate model
- Test other classifiers
- Create more features!

We've seen previously that data points were not well separated, even after engineering some numeric features and cleaning overlapping areas, it is inevitable that there will still be several adjacent points with no clear explanation because we are simply dealing with uninformative data thus classification is influenced by individual behavior which is highly unpredictable.

What about adding more features and using different classifier?

<li><a href="#toc">Table of contents</a></li>

<a id='mfnc'></a>

#### More features and a new classifier

RandomForest and [CatBoost](https://catboost.ai/) classifiers are [Ensembles Learners](https://en.wikipedia.org/wiki/Ensemble_learning) each using different method to generate predictions. RandomForest is a [Bootstrap aggregation - Bagging](https://en.wikipedia.org/wiki/Ensemble_learning#Bootstrap_aggregating_(bagging) classifier while CatBoost is, well, a [Boosting](https://en.wikipedia.org/wiki/Ensemble_learning#Boosting) one!

###### Ensemble Learning:

Ensemble methods employ a hierarchy of two algorithms; the low-level algorithm is called a base learner. The base learner is a single machine learning algorithm that gets used for all of the models that will get aggregated into an ensemble. The upper-level algorithm manipulates the inputs to the base learners so that the models they generate are somewhat independent.

###### RandomForest:

Random forests generate its sequence of models by training them on subsets of the data using decision trees as base learners. The subsets are drawn at random from the full training set, the first model is the first tree, the second model is the average of the first two trees, and the third model is the average of the first three trees and so on.

One way in which a subset is selected is to randomly sample rows with replacement (bootstrap) so an individual sample may be represented twice in that subset, another random element is that subsets don’t incorporate all the features rather a random subset of the features is also selected. So each tree will be trained on different training subset and using different combination of features used in previous trees and the conclusion reached by the most trees in the ensemble is considered to be the ensemble's overall assessment (aggregation).

###### CatBoost: 

CatBoost is a gradient boosting (GB) algorithm which develops an ensemble of tree based models, just like RandomForest. However, each new tree in the ensemble is trained on prediction errors (residuals) of earlier trees rather than training on different subsets of features and samples. We can think of GB as an optimization of the initial prediction by learning from previous errors hence the name Boosting; at the same time prediction errors(residuals) are slowly being reduced at each iteration taking small steps towards the correct predictions hence the name Gradient.

Each tree's contribution to the model is controlled by a value called Learning rate/step size, it scales a given tree's predictions by a small factor(value) that result in a small step in the right direction of minimizing prediction errors. If the steps are too large (using high value of learning rate) the optimization can diverge instead of converging (converge means that as the iterations proceed, the output gets closer and closer to optimal solution). If the step size is too small the process can take too many iterations, thus making the algorithm computationally expensive.

CatBoost is a nice off-the-shelf GB algorithm with default hyperparameter that produce good results, including automatically setting the learning rate. What is unique about CatBoost is the ability to handle categorical features without preprocessing, so you just mark the columns of these features and the algorithm will do the preprocessing steps which include target encoding but encoding is done [differently](https://github.com/catboost/catboost/blob/master/catboost/tutorials/categorical_features/categorical_features_parameters.ipynb) than how we did it earlier.

Another popular algorithms that handle categorical features directly is [LightGBM](https://lightgbm.readthedocs.io/), but we going to stick to CatBoost for now.

In [636]:
# Resampling the dataset into one hour interval

# convert time stamps cols back to datetime and extract scheduling year
df_split_re.scheduling_day = pd.to_datetime(df_split_re.scheduling_day)
df_split_re.appointment_day = pd.to_datetime(df_split_re.appointment_day)
df_split_re['sch_year'] = df_split_re.scheduling_day.dt.year

# build time series and aggregate statistics
ser = df_split_re.groupby(['scheduling_day']).no_show.sum().resample('h').sum().to_frame()

ser['total_apps'] = df_split_re.groupby(['scheduling_day']).no_show.count().resample('h').sum()

ser_ = df_split_re[['scheduling_day', 'patient_id']].copy()
ser_['rounded_hour'] = ser_.scheduling_day.dt.floor('h')

ser = pd.concat([ser, ser_.groupby('rounded_hour').patient_id.nunique()], axis=1).fillna(0)\
.rename(columns = {'patient_id':'patients_count'})

ser['absence_frequency'] = (ser.no_show / ser.total_apps).fillna(0)

ser.head()

In [637]:
# Filter on high absence frequency time slots, excluding the ones having few scheduled appointments 
ser[ser.total_apps > 2].sort_values(['absence_frequency', 'total_apps'], ascending=False).head(20)

In [638]:
# check neighborhood for a selected time slot
df_split_re[(df_split_re.sch_day == 25) & (df_split_re.sch_month == 2) & (df_split_re.sch_hour == 17) &
            (df_split_re.sch_year == 2016)].neighbourhood.unique()

In [639]:
# Neighborhood of another selected time slot
df_split_re[(df_split_re.sch_day == 25) & (df_split_re.sch_month == 4) & (df_split_re.sch_hour == 17) &
            (df_split_re.sch_year == 2016)].neighbourhood.unique()

Apparently scheduling time and day combined does impact 'no show' somehow specially that it is repeated among several different neighborhoods/patients and not an individual behavior, we've already seen similar pattern in timestamps earlier and maybe we can improve predictions creating some additional features within that context.

<li><a href="#toc">Table of contents</a></li>

<a id='upr'></a>

#### Updated prediction results: RandomForest and CatBoost

This section contains the final dataset of all features used in assessing classification performance for both RandomForest and CatBoost classifiers.

In [640]:
# Copies to modify
data = df_split_re.copy()

data.scheduling_day = pd.to_datetime(data.scheduling_day)
data.appointment_day = pd.to_datetime(data.appointment_day)

# More Features
data['sch_date'] = data.scheduling_day.dt.date.astype('str')
data['app_date'] = data.appointment_day.dt.date.astype('str')
data['sch_day_hour'] = data.scheduling_day.dt.floor('h').astype('str')

# Combine
data['nbr_sch_day'] = (data.neighbourhood + "_" + data.sch_date)
data['nbr_app_day'] = (data.neighbourhood + "_" + data.app_date)
data['nbr_sch_day_hour'] = (data.neighbourhood + "_" + data.sch_day_hour)
data['nbr_sms'] = (data.neighbourhood + "_" + data.sms_received.astype('str'))
data['nbr_age'] = (data.neighbourhood + "_" + data.age.astype('str'))

# To drop cat features laters
data.scheduling_day = data.scheduling_day.astype('str')
data.appointment_day = data.appointment_day.astype('str')

# Group unique samples
features = ['sch_date', 'sch_day_hour', 'app_date', 'nbr_sch_day', 'nbr_app_day', 'nbr_sch_day_hour', 'nbr_sms', 'nbr_age']

target = 'no_show'

combine_group(data, features, target, label_encode = True)

# Sort column
no_show_idx = data.columns.get_loc("no_show")

columns_list = data.columns.tolist()

columns_sort = columns_list[0:no_show_idx] + columns_list[no_show_idx+1:] + columns_list[no_show_idx : no_show_idx+1]

data = data[columns_sort]

data.head(2)

In [641]:
# update train/test sets with new features

# Copies to modify
X_train_mod_ = X_train_mod.copy()
X_test_mod_ = X_test_mod.copy()

new_feats = ['sch_date', 'sch_day_hour', 'app_date', 'nbr_sch_day', 'nbr_app_day', 'nbr_sch_day_hour', 'nbr_sms', 'nbr_age',
             'sch_date_', 'sch_day_hour_', 'app_date_', 'nbr_sch_day_', 'nbr_app_day_', 'nbr_sch_day_hour_', 'nbr_sms_', 'nbr_age_']

# update
X_train_mod_ = pd.concat([X_train_mod_, data[data.index.isin(X_train_mod_.index)][new_feats]], axis = 1, join= 'inner')
X_test_mod_ = pd.concat([X_test_mod_, data[data.index.isin(X_test_mod_.index)][new_feats]], axis = 1, join= 'inner')

X_train_mod_.head(1)

In [642]:
# Target Encode

# Copies to modify
X_train_mod_rf = X_train_mod_.copy()
X_test_mod_rf = X_test_mod_.copy()

# cols to encode
key = ['sch_date', 'sch_day_hour', 'app_date', 'nbr_sch_day', 'nbr_app_day', 'nbr_sch_day_hour', 'nbr_sms', 'nbr_age']

target_encode_feat_mapping(key, X_train_mod_rf, X_test_mod_rf, y_train_re, log_trans = True)

# Remove cat features features
X_train_mod_rf = X_train_mod_rf.drop(X_train_mod_rf.select_dtypes('object').columns, axis=1).copy()
X_test_mod_rf = X_test_mod_rf.drop(X_test_mod_rf.select_dtypes('object').columns, axis=1).copy()

X_test_mod_rf.head()

<a id='impr'></a>

In [643]:
rf_results_ = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), X_train_mod_rf, y_train_re, 
                                                                                             X_test_mod_rf, y_test)

In [ ]:
rf_plot(rf_results_.model, X_test_mod_rf)

Indeed the new features did improve classification performance and we can see couple of them at the top of the chart 'nbr_sch_day_hour_score' and 'nbr_age_score', us now at 75% and 68% for precision and recall compared to 68% and 64% previously.

<li><a href="#prevr">Earlier Results</a></li>

In [644]:
# Feature selection
ufs = UnivariateFeatureSelction(n_features = 20, problem_type = "classification", scoring = "mutual_info_classif")

features = X_train_mod_rf.columns

ufs.fit(X_train_mod_rf[features], y_train_re.values.ravel())

selected_features = ufs.return_cols(X_train_mod_rf[features])

selected_features

<a id='impr2'></a>

In [645]:
# Train on selected features only
X_train_mod_sf = X_train_mod_rf[selected_features]
X_test_mod_sf = X_test_mod_rf[selected_features]

rf_results_sf = train_eval(RandomForestClassifier(max_depth = 10, min_samples_leaf = 5, class_weight='balanced', random_state = 42), 
                           X_train_mod_sf, y_train_re, X_test_mod_sf, y_test)

Another improvement compared to earlier results, now we at 64% and 82% for precision and recall compared to 61% and 79%.

Note that the training data for RandomForest includes both label encoded and target encoded cat features, we also restricted the max depth to only 10 as we noted earlier that beyond that no improvement in results were noted.

Now we prepare the datasets for CatBoost classifier using only numeric and categorical features without any encodings.

<li><a href="#prevr2">Earlier Results</a></li>

In [646]:
# copies to modify
X_train_cat = X_train_mod_.copy()
X_test_cat = X_test_mod_.copy()

# remove target/label encoded cat feats
drop_cols = ['time_stamp_sch_', 'sch_time_', 'neighbourhood_', 'nbr_sch_time_', 'nbr_elapsed_days_', 'sch_date_', 
             'sch_day_hour_', 'app_date_', 'nbr_sch_day_', 'nbr_app_day_', 'nbr_sch_day_hour_', 'nbr_sms_', 'nbr_age_', 
             'time_stamp_sch_score', 'sch_time_score', 'neighbourhood_score', 'nbr_sch_time_score', 'nbr_elapsed_days_score',
             ]

X_train_cat.drop(drop_cols, inplace=True, axis=1)
X_test_cat.drop(drop_cols, inplace=True, axis=1)

# Mark cat columns
cat_cols_cat_boost = X_train_cat.select_dtypes('object').columns

X_train_cat.head(1)

In [647]:
# Check Cardinality of all cat features in both sets
for f in cat_cols_cat_boost:
    print(f'Cardinality of {f} in train set: {X_train_cat[f].nunique()}', '------', 
          f'and in test set: {X_test_cat[f].nunique()}')

In [649]:
# CatBoost

# creating Pools
train_set = cb.Pool(data = X_train_cat, label= y_train_re, cat_features = cat_cols_cat_boost.values)
eval_set = cb.Pool(data = X_test_cat, label= y_test, cat_features = cat_cols_cat_boost.values)

# Train
model = cb.CatBoostClassifier(random_seed = 42, early_stopping_rounds=5, one_hot_max_size = 24000, 
                              custom_metric = ['F1', 'Recall'])

model.fit(train_set, eval_set = eval_set, verbose = 50)

In [650]:
# Compare best scores for both sets 
model.get_best_score()

In [651]:
# Report full results - CatBoost default parameters using all features and OneHot encoding only

def scoring(model, features_test, labels_test, eval_set):
    
    # Hold predictions of +ve class
    pred_c = model.predict(features_test)
    pred_p = model.predict_proba(features_test)[:,1]

    # Display results
    pr_precision, pr_recall, pr_thresh = precision_recall_curve(labels_test, pred_p)
    pr_f1, pr_auc = f1_score(labels_test, pred_c), auc(pr_recall, pr_precision)

    curve = utils.get_roc_curve(model, eval_set)
    fpr, tpr, roc_thresh = curve
    roc_auc = auc(fpr, tpr)

    print('\n', '-' * 40, '\n', 'PR_AUC: ', pr_auc.round(4), '\n', '-' * 40, '\n', 'F1 score: ', pr_f1.round(4), '\n', '-' * 40, '\n',
          'ROC_AUC: ', roc_auc.round(4), '\n', '-' * 40, '\n', classification_report(labels_test, pred_c, 
                                                                                     target_names=['show', 'no_show']), 
          '-'*40, '\n', utils.get_confusion_matrix(model, eval_set))

scoring(model = model, features_test = X_test_cat, labels_test = y_test, eval_set = eval_set)

We've trained the model with original features only, the feature set include un-edited numeric and categorical features (no target encoding involved).

What’s unique about CatBoost is how it handles categorical features in raw format; no pre transformation to numeric values is needed as it is done internally upon fitting. First thing that CatBoost do is OneHot encoding cat features that include levels < the predefined threshold, no other encoding is applied to features that are OneHot encoded; Features that are not OneHot encoded are subject to series of target encoding and further combinations.

Our first attempt is to make CatBoost use one-hot encoding only for all our categorical features by setting the 'one_hot_max_size' hyperparameter to 24k as a threshold for applying OneHot encoding (the max categorical feature cardinality in our dataset is 23k). The documentation says, *Use one-hot encoding for all categorical features with a number of different values less than or equal to the given parameter value. Ctrs are not calculated for such features*. There is no clear definition of what does *Ctrs* stand for but let’s assume that it is 'Categorical to Ratios' acronym, as the provided equations in the documentation are a form of target encoding and our understanding for that parameter can also be confirmed [here](https://github.com/catboost/catboost/blob/master/catboost/tutorials/categorical_features/categorical_features_parameters.ipynb)

We've specified 'early_stopping_rounds', which is an overfitting detector hyperparameter, to be 5 this means that iterations stops when error rate of the test sets stalls for max 5 iterations while that of the train set keeps on decreasing (overfitting starts).

Setting 'custom_metric' to include f1 score and recall is for display purpose only, as no optimization for these two metrics are actually done during training the only metric optimize is the 'logloss' and that is what's displayed in each iteration results.

The takeaway from this attempt is that:
- OneHot encoding only did not provide good results compared to earlier ones
- Even when the model started overfitting early during training, we still got improvement in test results up to a specific number of trees (iterations) beyond which no improvement could be obtained.

In [652]:
# CatBoost Feature Importance
model.get_feature_importance(prettified=True)[:10]

Looking at top 10 features in terms of importance, there seems to be agreement between both CatBoost and RandomForest classifiers.

<a id='pm2'></a>

In [653]:
# Disable OneHot Encoding
model = cb.CatBoostClassifier(random_seed = 42, early_stopping_rounds=5, one_hot_max_size = 0,
                              custom_loss=['F1', 'Recall'])

model.fit(train_set, eval_set = eval_set, verbose = 50)

# Compare best socres for both sets
print('\n', '-' * 40, '\n', model.get_best_score())

scoring(model = model, features_test = X_test_cat, labels_test = y_test, eval_set = eval_set)

Results are just too good to be true! I'm usually skeptical when test results are too perfect, especially that we know that our model overfits.

The way train loss is higher than test loss suggests that we should revisit our train test split approach and/or train set design in general; remember that we resampled the training set with all samples of single appointment patients and also we are using a random 80:20 split.

Another thing to note is that CatBoost does combine categorical features that are not OneHotEncoded, our feature set already include some combinations of categorical features so this may also be contributing to the perfect scores we are getting.

So first thing to do is to reduce tree depth to 3 down from 6 the default parameter then we cancel further combinations of categorical features, will this affect our perfect scores?

<li><a href="#toc">Table of contents</a></li>

In [654]:
# adjusting tree depth
model = cb.CatBoostClassifier(random_seed = 42, early_stopping_rounds=5, one_hot_max_size = 0, depth = 3,
                              custom_loss=['F1', 'Recall'])

model.fit(train_set, eval_set = eval_set, verbose = 50)

# Compare best socres for both sets 
print('\n', '-' * 40, '\n', model.get_best_score())

scoring(model = model, features_test = X_test_cat, labels_test = y_test, eval_set = eval_set)

In [655]:
# growing small trees using all features and disabling further combinations

model = cb.CatBoostClassifier(random_seed = 42, one_hot_max_size = 0, depth = 3,
                              max_ctr_complexity = 0, custom_loss=['F1', 'Recall'])

model.fit(train_set, eval_set = eval_set, verbose = 50)

# Compare best scores for both sets 
print('\n', '-' * 40, '\n', model.get_best_score())

scoring(model = model, features_test = X_test_cat, labels_test = y_test, eval_set = eval_set)

In [656]:
# CatBoost Feature Importance
model.get_feature_importance(prettified=True)[:10]

So we pruned and restricted further categorical combinations, but results still seems to be too good especially that training set loss is higher than test set! How come we are good at predicting the unknown better than what we do at what we have learned?

It's because we still have leakage :D

Remember how these 'rare' levels are designed, we first combined each level with the corresponding target value then grouped combined levels having single sample into one category. Our aim was to create a more representative level per categorical feature in light of absence of sufficient +ve samples, there were two problems in such application:

- We combined and grouped features *before* train/test splits, so target values of latter split test set is leaked into training data. The classifier now knows that 'rare' levels have relatively high 'no show' rates which is ok if that was actually the case after doing the splits and transforming the test set not before, even after introducing noise (unseen categories assigned as being also 'rare').

- We are doing *random* train/test split so there is no guarantee that each level is equally represented in both sets for this approach to work correctly. This means that training set is more diverse thus its somewhat 'harder' to predict than the simpler test set version with few categories specially for the features with high importance, additionally, difference in sample size between both sets does affect loss calculation when it comes to gradient boosting.

One way to address this is to maintain a balanced and representative training set where categories are equally represented in both sets; this is actually really important and mostly overlooked as we are accustomed to train/test split without considering what actually goes into these sets.

We now need to pause and rethink our approach in light of what we know about the data so far. The most important thing we need to consider is our train/test split and define a *sampling unit*, we can either define our sampling unit to be a patient or time span.

Setting a sampling unit will dictate how to split the data and what features to use, if we are to use patients as a sampling unit then we would be interested in predicting the last appointment of each patient based on historical records, one problem of this approach is that we do not have sufficient historical records for each patient and there are many patients who have single appointments that will be excluded from our model; but this approach actually make sense in a real world application. We would normally expect patients to show up to their appointments, it’s after accumulating several appointments that we begin to rationalize and be able to predict patients' behavior.

If we decide to use time span as our sampling unit then we would also be interested in predicting the last appointment per each time slot, however, we know that there will be some gaps as our time series is not complete but bear in mind that we are using time features as a reflection of an underlying cause of 'no show' not the fact that it’s the exact time that is the cause of a 'no show'.

The choice of using last appoint as a test/validation set of whatever sampling unit we select is more intuitive, it doesn’t make sense to have random splits and include in the training set appointments from future dates to predict earlier appointments!

Train/test split, and training set design in general, is not always discussed or emphasized in most introductory machine learning guides. However, it’s of key importance when it comes to building and assessing predictive models. Now let's try both approaches.

<a id='rvapp'></a>

### Revised Approach

In this section we combine our knowledge gained from earlier attempts and approach the problem from different perspective, building two different models each capturing specific patterns contributing to the 'no show' problem.

In [657]:
# Time span as a sampling unit using full data

# Dropping duplicate/identical appointments
# Selecting columns to identify duplicates
mask = ml_df[['patient_id','gender','scheduling_day','appointment_day','age', 'neighbourhood','no_show']].copy() 

# Mark all +ve samples
mask['flag'] = mask.no_show.apply(lambda x: 1 if x ==1 else 0)

# Total +ve samples per group, used to capture identical appointments with different class labels
mask['flag_sum'] = mask.groupby(['patient_id', 'scheduling_day']).flag.transform('sum')

# -ve samples of duplicated appointments that have different class labels
# flag_sum >= 1 captures +ve samples per group and flag = 0 captures the -ve samples for which a +ve sample exists per the 
# same group
idx_1 = mask[(mask.flag_sum >=1) & (mask.flag ==0)].index

# duplicate observations to be dropped
idx_2 = mask[mask.duplicated()].index

# combined List of indices to be dropped
com_idx = idx_1.append(idx_2)

# applying filter
ml_df = ml_df[~ml_df.index.isin(com_idx)]

# reset index
ml_df.reset_index(inplace=True, drop=True)

# extracting additional time features
ml_df['time_stamp_sch'] = pd.to_datetime(ml_df.scheduling_day).dt.time.astype('str')
ml_df['sch_time'] = ml_df.time_stamp_sch.apply(lambda x: str(x)[:-3]).astype('str')

# separate categorical and numeric columns
cat_cols = ml_df.select_dtypes('object').columns
num_cols = ml_df.columns[~ml_df.columns.isin(cat_cols)]

# sort data by appointment date
ml_df = ml_df.sort_values(['appointment_day', 'scheduling_day']).copy()

ml_df.head()

In [658]:
# Train and test sets
# using last appointment of each time span as an input to test set
ml_df['last_app'] = ml_df.groupby(['sch_time', 'no_show']).appointment_id.transform('last')

test_set = ml_df[ml_df.appointment_id == ml_df.last_app].copy()
train_set = ml_df[~ml_df.index.isin(test_set.index)].copy()

# Group and combine then transform
feat = ['time_stamp_sch', 'sch_time']

for f in feat:
    # Combining best features with target
    train_set[f'{f}_target'] = (train_set[f'{f}'].astype(str) + '_' + train_set.no_show.astype(str))
    # number of samples per each combined category
    train_set[f'{f}_sample_count'] = train_set.groupby(f'{f}_target').no_show.transform('count')
    
    # statistics of unique samples (single samples)
    print(f'\'{f}\' unique train samples\' \'no_show\' mean:', round(train_set[train_set[f'{f}_sample_count'] == 1].no_show.mean(),2),\
          '---- sample count:', train_set[train_set[f'{f}_sample_count'] == 1].no_show.count())
    
    # grouping unique samples under one category
    train_set.loc[train_set[f'{f}_sample_count'] == 1, f'{f}_target'] = 'rare__'
    
    # update count
    train_set[f'{f}_sample_count'] = train_set.groupby(train_set[f'{f}_target'])['no_show'].transform('count')
    
    # Remove association with target
    train_set[f'{f}_target'] = train_set[f'{f}_target'].apply(lambda x: x[:-2])
    
    # mapping risk criteria to test set    
    mapr = dict(train_set[[f'{f}', f'{f}_target']].values)
    test_set[f'{f}'] = test_set[f'{f}'].map(mapr).fillna('rare')
    
    # drop temp column
    train_set[f'{f}'] = train_set[f'{f}_target']
    train_set.drop(f'{f}_target', axis = 1, inplace = True)

    print(f'\'{f}\' unique test samples\' \'no_show\' mean:', round(test_set[test_set[f'{f}'] == 'rare'].no_show.mean(),2),\
          '---- sample count:', test_set[test_set[f'{f}'] == 'rare'].no_show.count())
    
    # label Encode
    encoder = LabelEncoder()
    encoder.fit(train_set[f'{f}'])
    train_set[f'{f}_'] = encoder.transform(train_set[f'{f}'])
    test_set[f'{f}_'] = encoder.transform(test_set[f'{f}'])
    
# update categorical and numeric columns
cat_cols = train_set.select_dtypes('object').columns
num_cols = train_set.columns[~train_set.columns.isin(cat_cols)].drop(train_set.filter(regex = 'count').columns)

In [659]:
# Instantiate features/labels

feats = ['time_stamp_sch_', 'elapsed_days_sch']

X_train, y_train = train_set[feats], train_set.no_show
X_test, y_test = test_set[feats], test_set.no_show

# train and evaluate
rf_results_full = train_eval(RandomForestClassifier(class_weight='balanced', random_state = 42), X_train, y_train, 
                                                                                                 X_test, y_test)

So we've selected scheduling time to be our sampling unit and split the data in a way that ensure we are predicting last appointment of any given scheduling day using a single sample of each outcome (show/no show) per day. We've also created our rare categories but this time using train set as a lead while introducing noise categorizing unseen data as also a 'rare' category.

Finally we've used only two features being the exact scheduling time and day difference between scheduling and appointment dates to make predictions, this is to emphasize our earlier observations that time does influence no show somehow for this particular dataset.

Next we try a more logical and closer to a real world application approach using patient as a sampling unit.

In [660]:
# patient as a sampling unit using data of patients who have several records

# Encode user ID
encoder = LabelEncoder()
df_hist['patient_id_'] = encoder.fit_transform(df_hist.patient_id)

# use patients having several records
df_hist['app_count'] = df_hist.groupby('patient_id').no_show.transform('count')
df_hist = df_hist[df_hist.app_count >= 10].copy()

# switching threshold for marking an appointment as being tightly scheduled
# different thresholds affect prediction results
df_hist['same_day_count'] = df_hist[(df_hist.patient_id.duplicated(keep=False)) & (df_hist.schd_days_same ==1)]\
.sort_values(['patient_id', 'scheduling_day']).groupby(['patient_id', 'sch_day', 'sch_hour']).scheduling_day.transform('count')

df_hist['tight_schedule'] = np.where((df_hist.schd_seconds_diff > 0) & (df_hist.same_day_count > 1) & 
                                     (df_hist.schd_seconds_diff <= 411), 1,0)

# separate categorical and numeric columns
cat_cols = df_hist.select_dtypes('object').columns
num_cols = df_hist.columns[~df_hist.columns.isin(cat_cols)]

# Train and test sets split

# using last appointmentfor each patient as an input to test set
df_hist['last_app'] = df_hist.groupby('patient_id').appointment_id.transform('last')

test_set = df_hist[df_hist.appointment_id ==  df_hist.last_app].copy()

train_set = df_hist[~df_hist.index.isin(test_set.index)].copy()

# Mapping features to risk criteria
feat = ['patient_id_', 'age', 'neighbourhood', 'nbr_sch_time']

risk_lvl = ['rare', 'unlikely', 'possible', 'likely', 'certain', 'extreme']

for f in feat:

    # grouping levels into fewer risk criteria
    train_set[f'{f}_no_show_prlvl'] = train_set.groupby(f'{f}').no_show.transform('mean').round(3)
    train_set[f'{f}_risk_lvl'] = pd.cut(train_set[f'{f}_no_show_prlvl'], 6, labels = risk_lvl).astype('str')
    
    # statistics per risk criteria
    unique_labels = train_set[f'{f}_risk_lvl'].unique()
    
    for l in unique_labels:
        print(f'feature \'{f}\': ', f'Average \'no show\' of \'{l}\' risk rating :', round(train_set[train_set[f'{f}_risk_lvl'] == l].no_show.mean(),2),\
              '---- Total sample size :', train_set[train_set[f'{f}_risk_lvl'] == l].no_show.count())
        
    # mapping risk criteria to test set    
    mapr = dict(train_set[[f'{f}', f'{f}_risk_lvl']].values)
    test_set[f'{f}_risk_lvl'] = test_set[f'{f}'].map(mapr).fillna('rare')

    # Label Encode categories
    encoder = LabelEncoder()
    encoder.fit(train_set[f'{f}_risk_lvl'])
    train_set[f'{f}_risk_lvl_'] = encoder.transform(train_set[f'{f}_risk_lvl'])
    test_set[f'{f}_risk_lvl_'] = encoder.transform(test_set[f'{f}_risk_lvl'])
    
# update categorical and numeric columns
cat_cols = train_set.select_dtypes('object').columns
num_cols = train_set.columns[~train_set.columns.isin(cat_cols)].drop(train_set.filter(regex='count|prlvl|no_show').columns)

In [663]:
# Instentiate train/test featuers/labels

from random import shuffle

feats = ['sch_month', 'elapsed_days_sch', 'age_risk_lvl_', 'patient_id', 'M', 'age', 'patient_id__risk_lvl_', 'sms_received', 
         'same_day_app', 'tight_schedule']

X_train, y_train = train_set[feats], train_set.no_show
X_test, y_test = test_set[feats], test_set.no_show

# Target Encode
X_train_mod = X_train.copy()
X_test_mod = X_test.copy()

key = ['patient_id']

target_encode_feat_mapping(key, X_train_mod, X_test_mod, y_train, log_trans = True)

X_train_mod = X_train_mod.drop(key, axis=1).copy()
X_test_mod = X_test_mod.drop(key, axis=1).copy()

# train and evaluate
rf_results_patient = train_eval(RandomForestClassifier(class_weight = 'balanced', random_state = 42), X_train_mod, y_train, 
                                                                                                      X_test_mod, y_test)

We've created risk profiles for each patient and some of the features that contribute to no show, each portrays an underlying pattern that is captured based on historical no show behavior. We've also used different feature to train our model aiming to capture inconsistent behavior that we've noted earlier among patients.

We've selected patients having 10 appointments or more and split the train/test data using only last appoint of each patient as our validation set, thus using patient ID as a feature is justifiable as we are predicting appointments of same patients used to train the model.

You might be wondering why the features are in this exact order. I noticed really interesting phenomena that is the order of features does affect prediction results, so whenever I introduce a new feature I kept shuffling the feature list to obtain best possible combination.

Using cross validation for any of these approaches doesn't make sense in light of the available data we have.

<li><a href="#toc">Table of contents</a></li>

<a id='references'></a>

## Summary of updates and additional references

We've split the data into two datasets one for patients having historical(several) records and another for patients having single appointment, then we started analysing historical records of patients searching for 'no show' patterns and/or justifications in case of absence of common patterns.

Next, we proceeded with engineering numeric features based on our earlier analysis, mainly focusing on time details (hour, days, month, etc...) and time difference between appointments for each patient. We also created additional features to identify the nature of appointment (follow-up/substitute) and aggregated historical 'no show' behavior per patient, time slot and time slot/neighborhood combined.

The classifier did not find much value when trained on the above features as the initial (baseline) classification results were not satisfactory, however, based on the output of the classifier's feature importance and in-depth analysis of patient behavior we noted that time features does impact 'no show' behavior; examples of these features are the ones that capture differences between scheduling and appointment dates as well as differences between scheduling timing. Exact reason for why specific scheduling slots experienced higher 'no show' rates than others is still vague as there are some missing critical details for each appointment combined with the imbalanced nature of the data and lack of representative +ve class samples.

This led us to think about the 'no show' problem as a time series classification problem, however, this will impose some complications when it comes to segmenting the data and labeling each series for classification due to the reasons mentioned earlier. Alternatively, we've thought of creating categorical features that captures this 'no show' pattern, so we've dissected time stamps into different categorical feature each capturing different aspect of scheduling and appointment time and we also combined the feature 'neighborhood' with others such as 'scheduling hour', 'sms received' and 'age' to capture any significant contribution of these features combined to the 'no show' behavior. This approach follows our believe that there is an underlying reason at these specific dates that contribute to the 'no show' behavior and due to the absence of sufficient information we can’t conclude that specific reason but tried to capture it in the features we've created.

We've tried to present as much topics as possible that may be encountered going through a binary classification problem but with a twist!

### references:

Concept simplification:

- https://statquest.org/

- https://machinelearningmastery.com/

Publications, researches, thesis:

- [Adaptive Machine Learning for Credit Card Fraud Detection](http://www.ulb.ac.be/di/map/adalpozz/pdf/Dalpozzolo2015PhD.pdf)

- [A Comparison of Data Sampling Techniques for Credit Card Fraud Detection](https://thesai.org/Downloads/Volume11No6/Paper_60-A_Comparison_of_Data_Sampling_Techniques.pdf)

- [Employing transaction aggregation strategy to detect credit card fraud](http://euro.ecom.cmu.edu/resources/elibrary/epay/1-s2.0-S0957417412007166-main.pdf)

- [Overlap versus Imbalance](https://www.researchgate.net/publication/221442044_Overlap_versus_Imbalance)

- [Handling Class Overlap and Imbalance to Detect Prompt Situations in Smart Homes](https://eecs.wsu.edu/~cook/pubs/icdm13.2.pdf)

- [Classification with Class Overlapping: A Systematic Study](https://www.atlantis-press.com/article/2053.pdf)

- [Time Series Classification and its Applications](https://www.researchgate.net/publication/326248402_Time_Series_Classification_and_its_Applications)

- [Segmentation and Classification of Time-Series: Real Case Studies](https://www.researchgate.net/publication/221253019_Segmentation_and_Classification_of_Time-Series_Real_Case_Studies)

- [Calibrating Probability with Undersampling for Unbalanced Classification](https://www3.nd.edu/~dial/publications/dalpozzolo2015calibrating.pdf)

Books:

- [Machine Learning in Python: Essential Techniques for Predictive Analysis](https://www.wiley.com/en-ie/Machine+Learning+in+Python%3A+Essential+Techniques+for+Predictive+Analysis-p-9781119183600)

- [Imbalanced Learning: Foundations, Algorithms, and Applications](https://www.wiley.com/en-us/Imbalanced+Learning%3A+Foundations%2C+Algorithms%2C+and+Applications-p-9781118074626)

- [Approaching (Almost) Any Machine Learning Problem](https://github.com/abhishekkrthakur/approachingalmost)

walkthroughs:
- [Undersampling](https://machinelearningmastery.com/undersampling-algorithms-for-imbalanced-classification/)
- [Decision bounday visualization](https://www.kaggle.com/arthurtok/decision-boundaries-visualised-via-python-plotly/notebook)
- [Target Encoding](https://maxhalford.github.io/blog/target-encoding/)
- [Cuddling the Cats!](https://github.com/catboost/catboost/blob/master/catboost/tutorials/categorical_features/categorical_features_parameters.ipynb)

<li><a href="#toc">Table of contents</a></li>